In [0]:
import pyspark.sql.functions as F

In [0]:
df_airports = (
        spark.read.csv(
            "dbfs:/mnt/mids-w261/airport-codes_csv.csv",
            header=True,
            inferSchema=True
        )
    )

df_airports = (
    df_airports.withColumn(
        "long",
        F.split(F.col("coordinates"), ",")[0].cast("float")
    )
    .withColumn(
        "lat",
        F.split(F.col("coordinates"), ",")[1].cast("float")
    )
    .select("iata_code", "long", "lat")
    .dropna()
)

In [0]:



def transform_df(df):
    df = df.withColumn('FL_DATE', F.to_date('FL_DATE', 'yyyy-MM-dd'))
    df = df.withColumn(
            "hour",
            (F.col("CRS_DEP_TIME").cast("int") / F.lit(100)).cast("int")
        )
    
    return df

def ingest_data(dataset_path):
    # Read datasets from checkpoint
    train_df = spark.read.parquet(f"{dataset_path}/train.parquet").withColumn('FL_DATE', F.to_date('FL_DATE', 'yyyy-MM-dd'))
    train_df = transform_df(train_df)
    validation_df = spark.read.parquet(f"{dataset_path}/validation.parquet").withColumn('FL_DATE', F.to_date('FL_DATE', 'yyyy-MM-dd'))
    validation_df = transform_df(validation_df)
    test_df = spark.read.parquet(f"{dataset_path}/test.parquet")
    test_df = transform_df(test_df)
    
    return train_df, validation_df, test_df



In [0]:
def get_dataset_days(flight_df):
    dates = (
        flight_df
        .agg(
            F.min("FL_DATE").alias("min_date"),
            F.max("FL_DATE").alias("max_date")
        )
        .collect()[0]
    )
    
    min_date = dates["min_date"]
    max_date = dates["max_date"]
    
    n_days = (max_date - min_date).days + 1

    return n_days



In [0]:
import numpy as np
EARTH_RADIUS_M = 6_371_008.8

def haversine_np(lat1, lon1, lat2, lon2, *, radius=EARTH_RADIUS_M):
    """
    Vectorized great-circle distance.

    Inputs can be scalars or NumPy arrays (broadcastable to a common shape).
    Angles are in degrees. Returns distances in meters as float64.
    """
    lat1 = np.asarray(lat1, dtype=np.float64)
    lon1 = np.asarray(lon1, dtype=np.float64)
    lat2 = np.asarray(lat2, dtype=np.float64)
    lon2 = np.asarray(lon2, dtype=np.float64)

    # Convert degrees → radians in one pass
    φ1 = np.radians(lat1); λ1 = np.radians(lon1)
    φ2 = np.radians(lat2); λ2 = np.radians(lon2)

    dφ = φ2 - φ1
    dλ = λ2 - λ1

    s = np.sin(dφ * 0.5)
    t = np.sin(dλ * 0.5)
    a = s*s + np.cos(φ1) * np.cos(φ2) * t*t

    # Numerical guard ensures domain safety for asin
    return (2.0 * radius) * np.arcsin(np.minimum(1.0, np.sqrt(a)))/1000

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

def page_rank(flight_df, d=0.85, max_iter=20):
    # flights_df is your original DataFrame
    edges = (
        flight_df
        .groupBy("ORIGIN", "DEST")
        .agg(F.countDistinct("flight_uid").alias("weight"))  # or F.count("*")
        .withColumnRenamed("ORIGIN", "src")
        .withColumnRenamed("DEST", "dst")
    )


    # Sum weights by source
    out_sums = edges.groupBy("src").agg(F.sum("weight").alias("total_w"))

    # Normalize
    edges_norm = edges.join(out_sums, on="src") \
        .withColumn("norm_w", F.col("weight") / F.col("total_w"))

    # Get all distinct vertex IDs
    vertices = edges.select("src").union(edges.select("dst")).distinct()

    # Initialize PR = 1.0 for each node
    ranks = vertices.withColumn("page_rank", F.lit(1.0))


    page_ranks = page_rank_iter(edges_norm, ranks, d=d, max_iter=max_iter)

    return page_ranks.withColumnRenamed('src', 'ORIGIN')

def page_rank_iter(edges_norm, ranks, d=0.85, max_iter=20):

    for _ in range(max_iter):

        contribs = edges_norm.join(ranks, edges_norm['src'] == ranks['src']) \
                            .select(
                                edges_norm.dst.alias("id"),
                                (F.col("page_rank") * F.col("norm_w")).alias("contrib")
                            )
        new_ranks = contribs.groupBy("id").agg(F.sum("contrib").alias("sum_contrib"))
        ranks = new_ranks.select(
            col("id").alias("src"),
            ((1 - d) + d * F.col("sum_contrib")).alias("page_rank")
        )

    return ranks



In [0]:
# VERTEX
# out-degree
def out_degree(flight_df):
    return flight_df.groupBy("ORIGIN").agg(F.countDistinct("DEST").alias("out_degree"))

# in-degree
def in_degree(flight_df):
    flight_df = flight_df.groupBy("DEST").agg(F.countDistinct("ORIGIN").alias("in_degree"))
    return flight_df.withColumnRenamed("DEST", "ORIGIN")

# weighted out-degree (number of daily flights departing from an airport)


def weighted_in_out_degree(flight_df, DATASET_N_DAYS):
    in_degree = (
        flight_df
        .groupBy("DEST")
        .agg(
            (F.count('*') / F.lit(DATASET_N_DAYS)).alias("weighted_in_degree")
        )
        .withColumnRenamed("DEST", "ORIGIN")
    )

    out_degree = (
        flight_df
        .groupBy("ORIGIN")
        .agg(
            (F.count("*") / F.lit(DATASET_N_DAYS)).alias("weighted_out_degree")
        )
    )

    return in_degree.join(out_degree, on="ORIGIN")



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
csv_string = """ORIGIN,N_RUNWAYS
ORD,8
DFW,7
DTW,6
DEN,6
HNL,6
BOS,6
MWH,5
TCS,5
IAH,5
FST,5
AK51,5
MKE,5
ATL,5
41D,5
OKC,4
ILI,4
MIA,4
3PA2,4
OSH,4
NTU,4
LAX,4
LAS,4
MCO,4
CVG,4
PHL,4
IAD,4
CCR,4
SLC,4
CNM,4
SLN,4
CUH,4
MSP,4
NKT,4
MDW,4
JFK,4
PIL,4
BAC,4
AFF,4
PIT,4
MEM,4
VQQ,4
OAK,4
NQI,4
MWC,4
SVC,4
MU72,4
HLN,4
SFO,4
BZN,4
LHW,4
HDO,4
HWO,4
SER,4
NYL,4
GFK,4
SFB,4
BNA,4
MAF,4
DPA,4
SPS,4
FAI,4
NGP,4
STL,4
THA,4
NGS,4
2VA,3
C77,3
POY,3
POU,3
F06,3
SJT,3
COS,3
OSU,3
PSX,3
F01,3
VRB,3
MTO,3
LFT,3
ABQ,3
WAL,3
PTK,3
GLD,3
PUB,3
MO71,3
AAF,3
AUW,3
51G,3
F56,3
MRF,3
F45,3
44KS,3
MRH,3
MRI,3
PRC,3
SGJ,3
SJX,3
CPS,3
LBB,3
PRX,3
LAN,3
LAL,3
TUL,3
MSN,3
W03,3
PSC,3
JVL,3
LCG,3
F14,3
SEE,3
SEA,3
0KS5,3
SDF,3
BOW,3
GSO,3
MU97,3
NEL,3
APA,3
PDK,3
APC,3
ELM,3
PDX,3
ELP,3
N51,3
NFL,3
HYI,3
APF,3
EMM,3
PGD,3
BTL,3
ENA,3
END,3
NEW,3
SPF,3
ILG,3
PBI,3
EGQ,3
ANC,3
2H0,3
CYYJ,3
EFD,3
STP,3
68C,3
BWI,3
EKM,3
HWI,3
K34,3
PAQ,3
69V,3
SUA,3
BTR,3
NHK,3
NID,3
ENN,3
TYR,3
ADQ,3
NPA,3
LGB,3
EVV,3
NQX,3
C37,3
ARG,3
ARR,3
PMP,3
EWR,3
MVI,3
GRR,3
VGT,3
SLB,3
2B1,3
EVB,3
NMM,3
SMQ,3
6MY8,3
ENW,3
CVN,3
SUU,3
AIA,3
IND,3
EPH,3
HUT,3
GTB,3
MYF,3
PHX,3
6R5,3
SN29,3
6S8,3
6U5,3
PUC,3
K65,3
E11,3
MIC,3
TPA,3
RDU,3
MI4,3
DCA,3
FSD,3
RDK,3
OMA,3
OEB,3
DAY,3
ODO,3
HOB,3
BKE,3
BKD,3
MIT,3
HOU,3
TMB,3
BJC,3
ICT,3
DAB,3
FIN,3
FHU,3
RNO,3
RNP,3
IWA,3
LVM,3
FHB,3
02C,3
ROC,3
MHV,3
CMI,3
RYN,3
DEC,3
CLE,3
CLI,3
FOK,3
CL44,3
LSE,3
HIO,3
MEV,3
MDH,3
FPR,3
88NV,3
TAZ,3
LTS,3
FMM,3
24C,3
LQR,3
MCK,3
S44,3
FLY,3
LRU,3
MCI,3
CLT,3
ISW,3
X60,3
3R7,3
S60,3
ISP,3
FAR,3
LRD,3
SAF,3
DNV,3
RYM,3
RVS,3
WRL,3
CBM,3
HMN,3
FTT,3
BET,3
BIH,3
LWT,3
0C0,3
OPF,3
CHK,3
PWK,3
FCM,3
BIG,3
3JC,3
9D9,3
AYS,3
FWA,3
LIT,3
PWA,3
HFF,3
HRL,3
DYS,3
JRF,3
DLF,3
M47,3
IYK,3
YNG,3
CBK,3
MLI,3
LNA,3
SAT,3
MLB,3
DWH,3
AZO,3
FEP,3
BIL,3
CNO,3
SBA,3
00AR,3
ONY,3
MKO,3
LNK,3
SBN,3
FDR,3
BKX,2
BHM,2
OWI,2
OXB,2
OXI,2
54T,2
4NC2,2
BHB,2
IXD,2
BUF,2
OL37,2
P08,2
OKK,2
OWK,2
OL35,2
BKV,2
BIS,2
BIE,2
BKL,2
BTV,2
BKS,2
BTN,2
OZR,2
BJI,2
BTT,2
BTM,2
4P3,2
ITO,2
ITH,2
IWH,2
BKW,2
OLY,2
OWD,2
IOW,2
INW,2
INT,2
OOA,2
BRD,2
ONZ,2
50G,2
BMG,2
ORE,2
50D,2
ORF,2
ORG,2
ORH,2
ONT,2
4XS0,2
ONP,2
BMI,2
OPL,2
BRG,2
BPT,2
BNO,2
BNL,2
BNW,2
OR56,2
BOI,2
IPL,2
IPT,2
IRK,2
OQU,2
BUR,2
52B,2
51D,2
BPG,2
BPI,2
IRS,2
ISM,2
ORK,2
OLD,2
OLU,2
OLM,2
BLG,2
OTH,2
54J,2
4U6,2
OTM,2
OTZ,2
OUN,2
ISQ,2
OVE,2
OVO,2
OVS,2
OWA,2
OLE,2
OWB,2
BLH,2
OTG,2
ORL,2
OLZ,2
ONM,2
4WN4,2
BRL,2
4VG8,2
ONL,2
BLV,2
BRO,2
OMN,2
INS,2
4V9,2
INL,2
BLM,2
OME,2
4V1,2
INK,2
ZZV,2
C35,2
BUU,2
RCZ,2
CDC,2
RBW,2
IER,2
RCE,2
3W8,2
RCM,2
CDN,2
CDR,2
RCV,2
RCX,2
RDD,2
40C,2
RDG,2
3U8,2
CDS,2
3U3,2
CDV,2
CDW,2
IDP,2
3TX,2
3TR,2
CEC,2
CDB,2
RBE,2
CAE,2
PWD,2
CAK,2
PVD,2
CAO,2
42J,2
CAR,2
PVG,2
IGQ,2
PVU,2
PVW,2
CBE,2
PWG,2
RBD,2
PWM,2
CBF,2
IGM,2
PYM,2
PYX,2
41C,2
RAC,2
RAL,2
RAP,2
CCY,2
3TE,2
RFD,2
CEF,2
RPH,2
3N8,2
CGI,2
ROA,2
3MT4,2
CHA,2
CHD,2
ROW,2
ROX,2
RPB,2
RPD,2
3LF,2
3TA,2
RPX,2
RQB,2
RQO,2
RRL,2
RRT,2
RSL,2
3KK,2
RST,2
RSV,2
CHS,2
RNH,2
3NP,2
RND,2
CGC,2
RFI,2
CEK,2
RHI,2
RHV,2
RIC,2
RID,2
CEV,2
IDA,2
RIV,2
RIW,2
3RC,2
RKD,2
RKP,2
3R2,2
RKS,2
RLD,2
CFO,2
3OH6,2
RMG,2
CFS,2
CFV,2
PVB,2
CAD,2
P14,2
PIA,2
PGR,2
4D8,2
PGV,2
4D0,2
PHD,2
PHF,2
PHG,2
C04,2
PHN,2
PHP,2
PIE,2
PGA,2
PIH,2
PIR,2
C16,2
IMT,2
C20,2
49U,2
PKB,2
PKD,2
IMM,2
PKV,2
4E7,2
PEQ,2
CA89,2
BWP,2
BVS,2
BVU,2
BVX,2
P45,2
BVY,2
BWD,2
4M3,2
BWG,2
PAE,2
PAH,2
PAM,2
PEO,2
IN34,2
PBH,2
BXM,2
PBX,2
PCW,2
PCZ,2
PDC,2
PDT,2
BYI,2
4G1,2
IML,2
ILN,2
PLN,2
C91,2
IKV,2
PQN,2
PRB,2
PRG,2
C83,2
PRO,2
PRZ,2
C89,2
PSB,2
43Y,2
PSF,2
C29,2
IKK,2
43NV,2
IJX,2
PSN,2
IJD,2
PSP,2
43K,2
PTH,2
PTN,2
PTS,2
PQI,2
C81,2
PPO,2
PPG,2
49B,2
PMD,2
ILM,2
48S,2
C47,2
PNE,2
IL45,2
48D,2
48A,2
PNN,2
47V,2
PNS,2
PO1,2
POC,2
C73,2
IKW,2
POH,2
46NE,2
C75,2
PPA,2
C80,2
57AK,2
JLN,2
57C,2
7F3,2
LEE,2
9NE6,2
LEB,2
LDM,2
7G1,2
9R5,2
LCQ,2
MOB,2
MOD,2
MOP,2
LCK,2
MOT,2
LEW,2
7D3,2
7CO0,2
LCH,2
LBT,2
MPO,2
MPR,2
MPV,2
MPZ,2
MQB,2
MQI,2
MQJ,2
LEM,2
LEX,2
MLU,2
LIH,2
LLU,2
7S7,2
7S6,2
MML,2
LLN,2
MMT,2
MMU,2
MMV,2
LIU,2
MNM,2
MNN,2
LHX,2
LFK,2
9G8,2
9G9,2
LHV,2
9K7,2
9K8,2
LGU,2
7KS7,2
LGF,2
LGD,2
LGC,2
LGA,2
9U0,2
MQY,2
LBL,2
MVE,2
ABI,2
ABR,2
ABY,2
ACJ,2
ACK,2
72TA,2
ACT,2
ACV,2
ACY,2
ADC,2
MUT,2
ADG,2
MRC,2
ADH,2
MVN,2
70MO,2
MVY,2
MWA,2
MWL,2
6Y8,2
6Y6,2
ADM,2
MWO,2
MXF,2
ABE,2
73XA,2
AAT,2
MTW,2
79S,2
MRJ,2
LBF,2
MRT,2
MRY,2
9V9,2
78OH,2
LAR,2
78I,2
78A,2
MSL,2
MSO,2
MSS,2
MSY,2
77G,2
LAF,2
LAA,2
A14,2
L90,2
AA22,2
AAA,2
MTJ,2
74R,2
9A4,2
7T7,2
6XS2,2
93Y,2
MAL,2
MAW,2
8D9,2
LUV,2
8D7,2
8D3,2
8D1,2
MBG,2
MBL,2
LUM,2
MBS,2
LUK,2
MAI,2
MBY,2
8A1,2
LUF,2
LTY,2
89ME,2
MCG,2
MCN,2
MCW,2
LSV,2
MDD,2
94TT,2
LVK,2
93C,2
MLT,2
8M6,2
M19,2
M18,2
8XS8,2
M51,2
M54,2
LYH,2
8MO3,2
LXT,2
LXN,2
LXL,2
LWV,2
M73,2
LVS,2
M74,2
LWS,2
91TE,2
LWM,2
LWL,2
8K8,2
8K2,2
M80,2
92C,2
LWC,2
LWA,2
MDO,2
LSB,2
MDS,2
MJX,2
81D,2
LOL,2
80V,2
LNS,2
MIE,2
LNR,2
MIV,2
MIW,2
98XA,2
MJQ,2
80D,2
MKC,2
87Y,2
7Y9,2
MKG,2
LNN,2
7Y2,2
MKK,2
MKL,2
LNL,2
MKT,2
MLD,2
LMT,2
MLS,2
98G,2
MI58,2
MI55,2
98D,2
MDZ,2
87K,2
LS59,2
MEB,2
MEI,2
MEJ,2
MEZ,2
MFD,2
MFE,2
MFI,2
MGE,2
MGJ,2
MGM,2
MGR,2
LRF,2
MHE,2
MHK,2
MHL,2
MHR,2
MHT,2
LOU,2
LOT,2
MI45,2
6Y0,2
MXO,2
57D,2
AZ09,2
AVQ,2
O26,2
AWG,2
O29,2
JST,2
5ME2,2
AWO,2
AXA,2
AXN,2
JPX,2
JOT,2
JNU,2
AVP,2
JMS,2
JMR,2
AZ60,2
JKA,2
AZC,2
JHW,2
JHN,2
5K0,2
O88,2
B16,2
5I4,2
O22,2
AVO,2
NSE,2
NVD,2
NTD,2
5V8,2
NUI,2
NUQ,2
NUW,2
5U5,2
ASL,2
JXN,2
AST,2
ASW,2
ASX,2
NWL,2
AVK,2
ATA,2
ATS,2
ATW,2
ATY,2
AUG,2
JVY,2
NZY,2
AUO,2
5NK,2
AUS,2
JTC,2
OBE,2
OBI,2
OCF,2
BFD,2
OH86,2
59M,2
BDJ,2
BDL,2
BDR,2
BDU,2
BEA,2
JAN,2
BED,2
BEH,2
OIN,2
BFE,2
OCQ,2
BFF,2
BFI,2
58CL,2
BFL,2
BFM,2
BFR,2
IYS,2
BGD,2
57MT,2
BGE,2
BGM,2
BDH,2
BDE,2
BCV,2
JAX,2
OCW,2
ODX,2
OEL,2
OEO,2
BAF,2
5F1,2
OFK,2
BAK,2
BAM,2
5D5,2
BAX,2
BAZ,2
5C8,2
OGA,2
OGB,2
OGD,2
OGG,2
5B5,2
BBD,2
5B2,2
JEF,2
JCT,2
JBR,2
JYL,2
NRS,2
ADT,2
6D6,2
AIT,2
N15,2
6K3,2
AJG,2
6J6,2
AJZ,2
N35,2
N38,2
KTS,2
N58,2
KSM,2
N82,2
AIK,2
6B9,2
6B6,2
N85,2
AKN,2
NBC,2
NBG,2
6A1,2
69WT,2
NBJ,2
K91,2
NCA,2
AIO,2
MZZ,2
NRN,2
6R6,2
ADW,2
AEG,2
6V4,2
6V0,2
AEL,2
6U6,2
AEX,2
AFK,2
AFM,2
6S0,2
AFW,2
AGC,2
AIG,2
6R1,2
MYJ,2
AGS,2
AGZ,2
MYNN,2
AHN,2
AHQ,2
MYV,2
AIB,2
AID,2
6L3,2
K81,2
68LS,2
ND14,2
62D,2
NGW,2
NH64,2
APN,2
NIP,2
63C,2
NJK,2
APV,2
NK12,2
AQO,2
NKX,2
NLC,2
62C,2
ALB,2
NM10,2
AQP,2
61D,2
ARA,2
NM61,2
NOG,2
ARB,2
ART,2
ARV,2
JZI,2
JYR,2
NGT,2
NFD,2
65AR,2
AOO,2
ALI,2
ALM,2
ALN,2
ALO,2
K75,2
ALZ,2
AMA,2
AMN,2
67KY,2
AMW,2
NDZ,2
K63,2
K61,2
AND,2
66NE,2
ANE,2
K50,2
ANN,2
NE69,2
K44,2
ANW,2
ANY,2
65G,2
3K3,2
RDM,2
RTN,2
H34,2
EDW,2
19S,2
EED,2
EEN,2
19MN,2
EFC,2
U52,2
18V,2
EFK,2
EFT,2
EFW,2
GZH,2
U68,2
U69,2
EGV,2
U74,2
EHA,2
U78,2
GYY,2
EII,2
GYI,2
GXY,2
U93,2
U10,2
EDF,2
RTS,2
EDC,2
HAI,2
1G5,2
TWF,2
TWM,2
1G0,2
1F2,2
H81,2
TX22,2
EAG,2
1D7,2
1D6,2
TX42,2
EAR,2
1D1,2
EAU,2
1C9,2
TXK,2
EBS,2
1B9,2
TYS,2
ECG,2
1B0,2
U00,2
EKN,2
EKO,2
UAM,2
17G,2
GUC,2
UUU,2
GTU,2
14G,2
EQA,2
GTG,2
ERI,2
GTF,2
ERV,2
ERY,2
ESC,2
ESF,2
13K,2
ESN,2
EST,2
GTE,2
12WV,2
ETB,2
ETH,2
VAD,2
GST,2
EUG,2
12K,2
EOK,2
GUM,2
14MT,2
ULM,2
17FL,2
UBE,2
GWR,2
UCP,2
GWO,2
ELD,2
UES,2
UGN,2
UIN,2
GVL,2
ULS,2
UT94,2
ELY,2
15TT,2
EMP,2
UNK,2
15J,2
GUY,2
UNU,2
USE,2
ENL,2
ENV,2
1GA6,2
1H0,2
TCC,2
TIX,2
20IA,2
DNL,2
DNS,2
TKX,2
TLH,2
DOV,2
TMA,2
TMK,2
HLG,2
DRI,2
DRM,2
1V6,2
DSM,2
HKS,2
1U7,2
DTL,2
DTN,2
DTO,2
TNP,2
HJH,2
DUG,2
TOA,2
TOB,2
TKC,2
TIP,2
1H2,2
TIK,2
DEW,2
TCY,2
TDW,2
TDZ,2
DGW,2
22T,2
DHN,2
DHT,2
HNR,2
DIK,2
DKB,2
DKK,2
TE89,2
TEB,2
HND,2
DLH,2
DLL,2
DLN,2
DLS,2
21D,2
HMT,2
DMN,2
DMO,2
TOC,2
TOG,2
TOI,2
TOL,2
HEZ,2
E01,2
HEI,2
E06,2
1M6,2
HEG,2
HEF,2
HEE,2
HDL,2
E26,2
HDE,2
HDC,2
E38,2
E41,2
1K4,2
HBR,2
E57,2
TTN,2
TUS,2
1H8,2
TVC,2
TVF,2
TVI,2
HFD,2
DYR,2
1N5,2
HIB,2
TOP,2
TOR,2
TPF,2
TPH,2
TPL,2
TQK,2
TRI,2
TRK,2
TRM,2
DUX,2
DVK,2
DXX,2
DVL,2
DVN,2
TS70,2
DVT,2
HHF,2
HGR,2
TSG,2
1O3,2
DXR,2
1O1,2
12J,2
VBT,2
12G,2
08A,2
X26,2
X35,2
FLD,2
X51,2
GCK,2
FLL,2
X59,2
07F,2
FLO,2
GCD,2
GCC,2
FLX,2
FMH,2
06MN,2
FMN,2
FMY,2
06D,2
XLL,2
FMZ,2
XNA,2
GBG,2
XNO,2
FNL,2
X21,2
FKR,2
VCT,2
GDV,2
FDY,2
FFL,2
FFM,2
FFO,2
GEG,2
FFX,2
09MN,2
WVI,2
WVL,2
WWD,2
WWR,2
FFZ,2
GED,2
GE72,2
GE20,2
FJDG,2
08NY,2
GDW,2
FKL,2
X06,2
08K,2
X07,2
08D,2
FNT,2
FOA,2
FOD,2
FOE,2
Y91,2
Y93,2
Y94,2
Y95,2
Y98,2
01U,2
FTK,2
01OL,2
YAK,2
YIP,2
YKM,2
YKN,2
01G,2
FTW,2
FTY,2
FZY,2
FWC,2
FZG,2
FWS,2
FXE,2
FXY,2
ZER,2
ZPH,2
Y88,2
Y87,2
Y83,2
04KS,2
GBD,2
FPK,2
FPY,2
FRG,2
GAL,2
XWA,2
Y03,2
FRM,2
Y19,2
Y23,2
GAF,2
FSU,2
Y37,2
GAB,2
03WN,2
Y50,2
Y51,2
Y55,2
Y58,2
Y63,2
Y70,2
FSM,2
WST,2
WSN,2
GEY,2
F36,2
0W3,2
VNY,2
0V3,2
VPS,2
VPZ,2
F05,2
VSF,2
GRE,2
GRD,2
GRB,2
VTI,2
VTN,2
VUJ,2
VVS,2
VVV,2
VYS,2
GPZ,2
GPT,2
F21,2
GPI,2
W11,2
GOV,2
0Q3,2
VNW,2
VNC,2
GRI,2
11C,2
11Y,2
VCV,2
VDF,2
VDI,2
EVM,2
GSN,2
11LA,2
EVY,2
EWB,2
EWK,2
EWN,2
EZS,2
GSH,2
VHN,2
VIH,2
10IN,2
GRN,2
EZF,2
EZI,2
10C,2
VLA,2
VLD,2
W23,2
F37,2
FDK,2
GON,2
0G7,2
GGW,2
FAT,2
FAY,2
0F4,2
FBL,2
0E9,2
FBR,2
0E0,2
0D8,2
WLD,2
WLK,2
WLW,2
WMC,2
FBY,2
GGG,2
GGE,2
GFZ,2
GFL,2
0B8,2
0B4,2
WRI,2
GEZ,2
WDR,2
WDG,2
WBW,2
F82,2
GNV,2
W34,2
W41,2
F49,2
GNF,2
GNC,2
GMU,2
W78,2
F72,2
GLY,2
0L7,2
GHW,2
F85,2
GLS,2
GLR,2
GLH,2
GLE,2
WA25,2
F98,2
GKN,2
GJT,2
GIF,2
TCL,2
8V2,2
TBR,2
SMS,2
SLI,2
SLK,2
CSG,2
CSM,2
CSQ,2
SMD,2
SMF,2
CTB,2
SMX,2
SKX,2
CTK,2
CTY,2
2P2,2
SN18,2
2O7,2
2NK5,2
CVH,2
CVO,2
SLE,2
SKI,2
CVX,2
SGT,2
CPM,2
SFF,2
SFM,2
SFQ,2
SFZ,2
SGF,2
SGH,2
CPR,2
SHR,2
SJU,2
SHV,2
2VI2,2
2V6,2
CRG,2
CRP,2
CRS,2
SJC,2
SJN,2
CVS,2
CWA,2
CPF,2
STE,2
SRE,2
SRQ,2
SRR,2
SSC,2
SSF,2
SSI,2
CZBB,2
STC,2
CZD,2
2K1,2
STJ,2
STK,2
2GA2,2
STS,2
2G9,2
HWD,2
SUE,2
HVR,2
CYS,2
CYQG,2
HYX,2
HYA,2
SNA,2
CWI,2
HYS,2
2M8,2
SNK,2
HYR,2
SNS,2
SNY,2
HXF,2
SQI,2
CXO,2
SOW,2
SPG,2
HWV,2
SPI,2
CXY,2
SPW,2
2K7,2
SET,2
SEF,2
2F6,2
CKN,2
CKF,2
ICL,2
3DA,2
3D2,2
3D1,2
3CU,2
3CM,2
3C1,2
CKP,2
ICR,2
3B1,2
CKV,2
3A9,2
IAG,2
3A5,2
IAB,2
S33,2
CLK,2
S01,2
RZN,2
CLM,2
3GV,2
CHT,2
CIC,2
RUT,2
RVF,2
CID,2
3I3,2
CII,2
CIN,2
3GM,2
3F5,2
RWF,2
CIR,2
RWL,2
CIU,2
3G3,2
RYV,2
3F9,2
RZL,2
CLL,2
S39,2
SDY,2
I74,2
SBX,2
SBY,2
CO4,2
SCA,2
SCB,2
33V,2
SCH,2
SCK,2
COE,2
35D,2
SD25,2
COF,2
CON,2
COQ,2
SDA,2
32TT,2
SDM,2
COU,2
SBU,2
CNY,2
S43,2
37M,2
IA61,2
S64,2
CMH,2
CMX,2
CMY,2
38D,2
CN37,2
37XA,2
CNC,2
SBP,2
SAC,2
SAD,2
CNK,2
SAR,2
36K,2
SAV,2
SBM,2
CNW,2
SUS,2
SLH,2
HVE,2
HRX,2
HQG,2
SWO,2
2D1,2
D25,2
D95,2
2D5,2
HTL,2
SVE,2
HOT,2
D60,2
DCY,2
D39,2
SYF,2
D90,2
TA54,2
DDC,2
HOP,2
SWT,2
HON,2
D37,2
HUF,2
D05,2
29KS,2
DAG,2
SWD,2
D09,2
DAL,2
HPN,2
SYN,2
SYR,2
2B3,2
D61,2
DCM,2
DAN,2
HUL,2
HUM,2
DBN,2
HTH,2
DBQ,2
T41,2
HSV,2
DED,2
T00,2
DEE,2
HSB,2
28J,2
TAN,2
D50,2
D55,2
27C,2
HSI,2
T12,2
SUW,2
D17,2
T69,2
28X,2
SXQ,2
SXU,2
SUX,2
HSR,2
SWF,2
SWW,2
TAD,2
LMO,1
LL81,1
LL22,1
IA31,1
IA32,1
IA18,1
GE99,1
LL10,1
GEV,1
LKV,1
IA38,1
LKU,1
LLJ,1
LKR,1
LKP,1
IA12,1
LNP,1
GE61,1
GEO,1
GE56,1
HRI,1
LKK,1
GEU,1
IA26,1
HRJ,1
JAQ,1
IA24,1
JAS,1
HLX,1
IEN,1
HRR,1
HRO,1
IA20,1
HRT,1
LND,1
IEM,1
HLC,1
LJF,1
IFP,1
LLR,1
LMS,1
LLQ,1
GFD,1
IA11,1
IA07,1
HLB,1
IA21,1
LNC,1
IFA,1
IA05,1
GGF,1
LCI,1
LBR,1
LBX,1
II93,1
I73,1
GLW,1
GLV,1
HSA,1
JDN,1
I75,1
I76,1
IGT,1
JER,1
GKY,1
GKT,1
IIB,1
HJO,1
JES,1
GMJ,1
I72,1
GNB,1
I71,1
IIK,1
LBO,1
GNG,1
IIY,1
GNT,1
GNU,1
HSD,1
HII,1
GOK,1
HSE,1
LDJ,1
GKJ,1
IG04,1
HKY,1
IA02,1
LHD,1
LHM,1
LHQ,1
IA42,1
HKA,1
HRU,1
I80,1
LHZ,1
LIC,1
JAU,1
IGG,1
HLA,1
IG05,1
GGI,1
LHB,1
I99,1
GGP,1
I96,1
JCA,1
GGV,1
I95,1
I93,1
GHG,1
I91,1
LFI,1
I86,1
GHM,1
JDD,1
GIC,1
I83,1
ISB,1
GCY,1
HRF,1
ID8,1
GA46,1
LUX,1
LVJ,1
ID31,1
ID70,1
ID67,1
GA36,1
LVL,1
ID39,1
LVN,1
ID40,1
LWB,1
LWD,1
IWD,1
HNS,1
GA49,1
ID06,1
LUR,1
HNH,1
LUA,1
LUD,1
LUG,1
IXA,1
GAD,1
IWS,1
ID76,1
GA65,1
LUL,1
IWK,1
LUP,1
HNM,1
ID75,1
IWI,1
ID45,1
ID63,1
HOM,1
M12,1
M02,1
M04,1
M05,1
ID50,1
M08,1
M11,1
M13,1
HOE,1
M15,1
M17,1
FYU,1
M20,1
M21,1
M22,1
M01,1
LZZ,1
IUA,1
HNZ,1
ID57,1
GA19,1
G05,1
FZI,1
ID52,1
LXV,1
LXY,1
LZU,1
FYV,1
LYO,1
LYV,1
ID51,1
LZD,1
ITR,1
GAG,1
HPB,1
IA43,1
IYA,1
LPR,1
LQK,1
GCT,1
GCN,1
GCM,1
IA68,1
HQM,1
IA76,1
IA77,1
IZG,1
LRG,1
IDL,1
LRJ,1
HMP,1
IDI,1
LPC,1
LOZ,1
IA63,1
GDM,1
M23,1
IA51,1
HQZ,1
JAC,1
IA59,1
LNY,1
HQU,1
HSG,1
LOM,1
GDK,1
GDJ,1
GDB,1
GDA,1
IA62,1
LRO,1
GBR,1
LRY,1
ID83,1
IAN,1
HMZ,1
ID89,1
GAS,1
GAO,1
HNB,1
LSF,1
LS87,1
GAM,1
LSK,1
LSN,1
ISO,1
IBM,1
GAI,1
LS88,1
LS84,1
LS08,1
IZA,1
GBH,1
IA83,1
IA94,1
LS09,1
IDG,1
LS21,1
ID97,1
ID90,1
HPY,1
HPT,1
LS26,1
LS46,1
ID93,1
LS70,1
I69,1
HSP,1
LBE,1
JRO,1
HWY,1
IN3,1
IN29,1
HXD,1
INJ,1
K67,1
JSV,1
K68,1
K74,1
GYR,1
K78,1
GYL,1
HEQ,1
GYH,1
GYB,1
K62,1
GZL,1
K59,1
JSY,1
IN4,1
K38,1
H04,1
IN37,1
K43,1
GZS,1
K49,1
K57,1
INF,1
IN30,1
K51,1
K52,1
HDN,1
GZN,1
JSO,1
K79,1
HFJ,1
IN26,1
KGX,1
KIC,1
KKA,1
HYW,1
JQF,1
GWB,1
JQD,1
KLG,1
KLS,1
KNB,1
KNW,1
KOA,1
KPC,1
GVT,1
KTN,1
KFP,1
IN16,1
KEK,1
K94,1
K82,1
K83,1
K88,1
IN21,1
K89,1
HES,1
K96,1
KEB,1
KAL,1
IN2,1
GXF,1
GWW,1
GWS,1
KCL,1
H05,1
HWQ,1
K33,1
HDH,1
JZP,1
JZZ,1
HAB,1
H97,1
H96,1
K01,1
H92,1
H88,1
K06,1
IN6,1
JWN,1
H80,1
H79,1
HVC,1
H71,1
IN7,1
JYO,1
JYM,1
HAO,1
HBE,1
JY24,1
HVN,1
JXI,1
HBG,1
JY73,1
IN82,1
IN75,1
HVS,1
IN9,1
HBI,1
HAF,1
JWY,1
HAE,1
K07,1
K08,1
HBV,1
K26,1
HCO,1
K24,1
JVW,1
H57,1
IN41,1
H47,1
H37,1
K22,1
K29,1
H35,1
HCR,1
HUS,1
H19,1
H17,1
HCD,1
HCA,1
K09,1
IN51,1
K11,1
K15,1
HBZ,1
IN58,1
IN55,1
H70,1
IN50,1
H58,1
IN5,1
JWG,1
H68,1
K17,1
K19,1
H63,1
GVQ,1
KVC,1
GOO,1
L70,1
HIE,1
L61,1
I42,1
I44,1
L62,1
I45,1
I50,1
I54,1
GRF,1
I61,1
L65,1
L66,1
I63,1
I64,1
HIF,1
I40,1
GRK,1
I35,1
GSB,1
JFZ,1
I23,1
JFX,1
IL28,1
I27,1
IL22,1
L39,1
L52,1
IL05,1
GRO,1
L41,1
HTF,1
I34,1
L45,1
HIG,1
L71,1
HZD,1
L72,1
LA90,1
IKO,1
GQQ,1
HBC,1
IOB,1
I67,1
GPM,1
LAM,1
I68,1
GPH,1
IPJ,1
GPC,1
LAW,1
IKG,1
GOP,1
LA89,1
LA52,1
I66,1
HST,1
L73,1
L77,1
L78,1
L80,1
L83,1
L88,1
L92,1
LA46,1
LA03,1
LA04,1
LA11,1
LA17,1
LA35,1
LA41,1
L37,1
IL47,1
I22,1
IL48,1
L09,1
HZR,1
IN07,1
GUS,1
HUA,1
HZX,1
GUR,1
IN01,1
HZY,1
I00,1
GUP,1
HTW,1
IN0,1
L17,1
L18,1
L08,1
L06,1
IN09,1
KYK,1
KVL,1
KWA,1
KWT,1
KY8,1
JNX,1
IN12,1
KYU,1
HZL,1
L00,1
GVE,1
HFY,1
L04,1
L05,1
HZE,1
JLA,1
GTR,1
L19,1
L36,1
GSZ,1
L33,1
JHM,1
HHW,1
L35,1
I18,1
JGG,1
HOC,1
I20,1
ILE,1
GSP,1
IL91,1
IL78,1
IL54,1
I17,1
HHR,1
L20,1
HTS,1
I01,1
IMS,1
JKL,1
JKJ,1
I06,1
I10,1
L22,1
L32,1
L25,1
L26,1
I16,1
HHG,1
L31,1
HHI,1
JYG,1
N47,1
M24,1
T30,1
T50,1
T47,1
T45,1
T42,1
T38,1
T37,1
T36,1
T35,1
T29,1
SZY,1
T28,1
T23,1
T20,1
T16,1
T15,1
T14,1
T11,1
T05,1
T51,1
T54,1
T56,1
T59,1
TA24,1
TA20,1
TA12,1
T96,1
T93,1
T92,1
T90,1
T89,1
T88,1
T85,1
T82,1
T80,1
T79,1
T78,1
T74,1
T70,1
T60,1
T03,1
SZT,1
TA27,1
SPK,1
STT,1
STF,1
SSQ,1
SRV,1
SRC,1
SRB,1
SQL,1
SPZ,1
SPH,1
SZP,1
SPB,1
SPA,1
SOV,1
SOP,1
SOA,1
SNP,1
SNL,1
SNH,1
STX,1
SUD,1
SUN,1
SUO,1
SZL,1
SYV,1
SYM,1
SYI,1
SYA,1
SXP,1
SXL,1
SXK,1
SWI,1
SVW,1
SVS,1
SVR,1
SVN,1
SVH,1
SVA,1
SUZ,1
SUT,1
TA25,1
TA28,1
M25,1
TN68,1
TNI,1
TNCM,1
TNC,1
TN99,1
TN95,1
TN87,1
TN81,1
TN75,1
TN54,1
TLJ,1
TN47,1
TN45,1
TN41,1
TN20,1
TN09,1
TMT,1
TME,1
TLT,1
TNT,1
TNU,1
TNW,1
TNX,1
TSO,1
TS95,1
TS90,1
TS87,1
TS80,1
TS76,1
TS75,1
TS47,1
TS44,1
TS41,1
TS20,1
TS07,1
TS05,1
TRX,1
TRL,1
TQH,1
TQE,1
TLR,1
TKV,1
TA35,1
TBN,1
TE29,1
TE25,1
TE05,1
TE02,1
TDO,1
TDF,1
TCT,1
TCM,1
TAL,1
TKK,1
TA95,1
TA90,1
TA85,1
TA77,1
TA68,1
TA66,1
TA60,1
TA56,1
TE35,1
TE38,1
TE39,1
TE44,1
TKI,1
TKA,1
TIW,1
TIF,1
THV,1
THM,1
TGI,1
TGC,1
TFP,1
TEX,1
TEW,1
TER,1
TEL,1
TE85,1
TE73,1
TE58,1
TE54,1
SNC,1
SN88,1
SN83,1
S30,1
S42,1
S40,1
S37,1
S36,1
S35,1
S34,1
S32,1
S31,1
S28,1
S05,1
S25,1
S24,1
S23,1
S21,1
S20,1
S19,1
S16,1
S12,1
S45,1
S50,1
S51,1
S52,1
S94,1
S93,1
S89,1
S87,1
S85,1
S83,1
S81,1
S78,1
S75,1
S73,1
S72,1
S71,1
S70,1
S69,1
S68,1
S66,1
S59,1
S10,1
S03,1
SN68,1
RMP,1
ROR,1
ROI,1
ROG,1
RNV,1
RNT,1
RNM,1
RNC,1
RMY,1
RMN,1
RZT,1
RME,1
RKW,1
RKR,1
RK1,1
RJD,1
RIU,1
RIR,1
RIL,1
ROS,1
RPJ,1
RQE,1
RRQ,1
RZR,1
RYY,1
RYW,1
RXE,1
RWV,1
RWN,1
RWI,1
RVR,1
RVN,1
RVL,1
RVJ,1
RUQ,1
RUG,1
RUE,1
RSW,1
RSN,1
RSH,1
S95,1
S97,1
SAA,1
SIY,1
SLR,1
SLQ,1
SLO,1
SLG,1
SKW,1
SKF,1
SKA,1
SJS,1
SIV,1
SAN,1
SIT,1
SIK,1
SIG,1
SIF,1
SHX,1
SHN,1
SHL,1
SHH,1
SME,1
SMK,1
SMN,1
SMO,1
SN65,1
SN64,1
SN61,1
SN55,1
SN52,1
SN51,1
SN47,1
SN45,1
SN42,1
SN40,1
SN33,1
SN32,1
SN22,1
SN20,1
SN13,1
SN10,1
SMU,1
SHG,1
SHD,1
SH1,1
SCR,1
SCD,1
SCC,1
SC99,1
SC86,1
SC84,1
SC67,1
SC66,1
SC58,1
SC36,1
SC07,1
SC05,1
SBS,1
SBO,1
SBD,1
SAZ,1
SAW,1
SAS,1
SCM,1
SCX,1
SGY,1
SD15,1
SGU,1
SGS,1
SGR,1
SFY,1
SEZ,1
SEP,1
SEM,1
SEG,1
SDP,1
SDL,1
SDC,1
SD77,1
SD50,1
SD32,1
SD29,1
SD27,1
SD2,1
TSP,1
TT11,1
TT12,1
WI46,1
WN19,1
WMO,1
WJF,1
WI98,1
WI80,1
WI72,1
WI66,1
WI52,1
WI33,1
WA87,1
WI31,1
WHP,1
WEA,1
WCR,1
WBQ,1
WBB,1
WAY,1
WA97,1
WN22,1
WN28,1
WN29,1
WN53,1
WV95,1
WV68,1
WV12,1
WTK,1
WT44,1
WT37,1
WT32,1
WT18,1
WT15,1
WT03,1
WSM,1
WS84,1
WS51,1
WS20,1
WRG,1
WRB,1
WNA,1
WA93,1
WA76,1
VTA,1
W20,1
W40,1
W35,1
W31,1
W29,1
W28,1
W27,1
W24,1
W22,1
W16,1
WA67,1
W13,1
W10,1
W09,1
W05,1
W04,1
W01,1
W00,1
VWU,1
W42,1
W43,1
W48,1
W50,1
WA45,1
WA39,1
WA20,1
WA14,1
W99,1
W96,1
W95,1
W94,1
W91,1
W90,1
W87,1
W81,1
W75,1
W63,1
W57,1
W56,1
W52,1
WY00,1
WY02,1
WY08,1
XVG,1
Y71,1
Y65,1
Y49,1
Y47,1
Y43,1
Y31,1
Y27,1
Y14,1
XSA,1
WY19,1
XS93,1
XS76,1
XS75,1
XS71,1
XS67,1
XS46,1
XS44,1
XS42,1
Y72,1
Y73,1
Y74,1
Y77,1
ZNC,1
ZEF,1
Z99,1
Z98,1
Z91,1
Z90,1
Z84,1
Z81,1
Z55,1
Z47,1
Z40,1
Z17,1
Z13,1
Z09,1
Z08,1
Y99,1
Y89,1
XS36,1
XS22,1
XS16,1
X39,1
X33,1
X25,1
X23,1
X16,1
X14,1
X13,1
X10,1
X09,1
X04,1
X01,1
WYS,1
WY58,1
WY56,1
WY53,1
WY46,1
WY45,1
WY41,1
X36,1
X44,1
XS09,1
X50,1
XS05,1
XNX,1
XNI,1
XMR,1
XBP,1
XA89,1
XA81,1
XA66,1
XA49,1
XA45,1
XA42,1
XA03,1
XA0,1
X65,1
X63,1
X58,1
X55,1
VUO,1
VT8,1
TT16,1
TXW,1
U03,1
U02,1
U01,1
TZV,1
TZT,1
TZR,1
TYQ,1
TYL,1
TXKF,1
TX14,1
TX94,1
TX90,1
TX82,1
TX40,1
TX36,1
TX32,1
TX25,1
TX2,1
U05,1
U08,1
U12,1
U14,1
U76,1
U70,1
U65,1
U64,1
U62,1
U61,1
U58,1
U56,1
U55,1
U53,1
U48,1
U46,1
U41,1
U37,1
U36,1
U34,1
U30,1
TX16,1
TX13,1
VT76,1
TT37,1
TT53,1
TT50,1
TT48,1
TT47,1
TT46,1
TT44,1
TT41,1
TT40,1
TT35,1
TX0,1
TT34,1
TT31,1
TT29,1
TT27,1
TT26,1
TT21,1
TT19,1
TT18,1
TT57,1
TT65,1
TT66,1
TT73,1
TWT,1
TVY,1
TVR,1
TVL,1
TVK,1
TVB,1
TUP,1
TTS,1
TTK,1
TTF,1
TTD,1
TTA,1
TT99,1
TT98,1
TT91,1
TT87,1
TT75,1
U81,1
U82,1
U86,1
VA89,1
VEE,1
VDZ,1
VCB,1
VBW,1
VBG,1
VAY,1
VAK,1
VA99,1
VA65,1
U87,1
VA63,1
VA59,1
VA44,1
VA42,1
VA34,1
VA26,1
VA20,1
VA10,1
VEL,1
VER,1
VES,1
VG18,1
VT72,1
VQS,1
VPC,1
VOK,1
VMR,1
VLL,1
VKX,1
VKS,1
VJI,1
VIS,1
VIQ,1
VGC,1
VG67,1
VG65,1
VG54,1
VG31,1
VG25,1
UZA,1
UYF,1
UXL,1
UKT,1
UKI,1
UKF,1
UIL,1
UDG,1
UDD,1
UCY,1
UBX,1
UBW,1
UBS,1
UAO,1
U99,1
U96,1
U94,1
U92,1
U91,1
U89,1
U88,1
UKL,1
UMM,1
UWL,1
UMP,1
UVA,1
UUV,1
UUO,1
UTS,1
UTO,1
UTA,1
UT91,1
UT9,1
UT89,1
UT53,1
USW,1
UPP,1
UOX,1
UNV,1
UNO,1
UNI,1
UMT,1
RIF,1
RHP,1
RGK,1
MUE,1
MY17,1
MXA,1
MWM,1
MWK,1
MVM,1
MVL,1
MVC,1
MUO,1
MU99,1
MU26,1
MU98,1
MU89,1
MU88,1
MU83,1
MU73,1
MU68,1
MU62,1
MU57,1
MY43,1
MY63,1
MY88,1
MYAB,1
MYLD,1
MYL,1
MYIG,1
MYGF,1
MYER,1
MYEM,1
MYEH,1
MYEF,1
MYCB,1
MYBS,1
MYBG,1
MYAT,1
MYAP,1
MYAN,1
MYAM,1
MYAK,1
MYAF,1
MU38,1
MU22,1
MS20,1
MT15,1
MT56,1
MT49,1
MT46,1
MT36,1
MT34,1
MT26,1
MT24,1
MT22,1
MT13,1
MU14,1
MSV,1
MS92,1
MS9,1
MS88,1
MS62,1
MS51,1
MS48,1
MS44,1
MT57,1
MT61,1
MT62,1
MT63,1
MU13,1
MU06,1
MTV,1
MTP,1
MTN,1
MTH,1
MTC,1
MT96,1
MT92,1
MT89,1
MT86,1
MT84,1
MT82,1
MT81,1
MT73,1
MT68,1
MT66,1
MYLS,1
MYR,1
MYRP,1
ND19,1
ND82,1
ND64,1
ND58,1
ND54,1
ND43,1
ND40,1
ND38,1
ND26,1
ND18,1
MYSM,1
ND12,1
ND10,1
ND08,1
ND07,1
ND02,1
NC76,1
NC74,1
NC44,1
ND83,1
ND84,1
ND92,1
NDY,1
NE82,1
NE81,1
NE73,1
NE67,1
NE53,1
NE51,1
NE49,1
NE45,1
NE43,1
NE36,1
NE33,1
NE30,1
NE26,1
NE25,1
NE24,1
NE15,1
NE12,1
NC35,1
NC01,1
NA13,1
N43,1
N30,1
N29,1
N27,1
N24,1
N23,1
N19,1
N17,1
N14,1
N13,1
N12,1
N07,1
N06,1
N03,1
MZJ,1
MZH,1
MYZ,1
MYU,1
N40,1
FYJ,1
NA07,1
N49,1
N98,1
N96,1
N94,1
N89,1
N87,1
N86,1
N81,1
N79,1
N74,1
N71,1
N69,1
N68,1
N66,1
N63,1
N57,1
N56,1
N52,1
MS41,1
MRN,1
NE85,1
MCB,1
MD31,1
MD1,1
MCZ,1
MCX,1
MCF,1
MCE,1
MCD,1
MCC,1
MBT,1
MAE,1
MBSC,1
MBPV,1
MBO,1
MBNC,1
MBA,1
MAZ,1
MAO,1
MAN,1
MD84,1
MDF,1
MDM,1
MDQ,1
MHM,1
MGY,1
MGW,1
MGN,1
MGG,1
MGC,1
MFV,1
MFR,1
MEY,1
MER,1
ME84,1
ME55,1
ME5,1
ME2,1
ME16,1
MDY,1
MDT,1
MAJ,1
MAC,1
MRB,1
M40,1
M53,1
M50,1
M48,1
M46,1
M45,1
M44,1
M43,1
M41,1
M38,1
M99,1
M37,1
M36,1
M34,1
M32,1
M30,1
M29,1
M28,1
M27,1
M55,1
M56,1
M57,1
M59,1
M95,1
M94,1
M91,1
M90,1
M86,1
M85,1
M83,1
M79,1
M78,1
M77,1
M75,1
M72,1
M71,1
M70,1
M69,1
M66,1
M60,1
MHP,1
MI1,1
MI10,1
MO23,1
MO46,1
MO43,1
MO39,1
MO33,1
MO31,1
MO30,1
MO3,1
MO29,1
MO22,1
MI17,1
MO2,1
MO18,1
MO17,1
MO15,1
MO11,1
MO1,1
MO06,1
MO04,1
MO48,1
MO49,1
MO5,1
MO51,1
MQW,1
MQS,1
MPJ,1
MPI,1
MPG,1
MPE,1
MOX,1
MOU,1
MOR,1
MO95,1
MO9,1
MO89,1
MO85,1
MO8,1
MO74,1
MO66,1
MO61,1
MO01,1
MO00,1
MNZ,1
MLE,1
MKY,1
MKV,1
MKS,1
MKN,1
MKJ,1
MKA,1
MJD,1
MIO,1
MIB,1
MI93,1
MI83,1
MI79,1
MI69,1
MI47,1
MI39,1
MI37,1
MI31,1
MLC,1
MLF,1
MNV,1
MLJ,1
MNI,1
MNF,1
MNE,1
MN98,1
MN73,1
MN71,1
MN46,1
MN44,1
MN26,1
MN24,1
MN21,1
MN09,1
MMS,1
MMK,1
MMI,1
MMH,1
MLY,1
NE83,1
NE89,1
RGA,1
OZW,1
P19,1
P17,1
P15,1
P13,1
P10,1
P04,1
P03,1
P01,1
OZS,1
OUL,1
OZA,1
OYM,1
OXV,1
OXR,1
OXD,1
OXC,1
OWX,1
OWP,1
P20,1
P23,1
P29,1
P33,1
PEX,1
PEV,1
PEA,1
PCM,1
PCD,1
PBV,1
PBG,1
PBF,1
PAO,1
PAN,1
PAK,1
PA88,1
PA44,1
P96,1
P91,1
P52,1
P34,1
OVL,1
OSX,1
OL55,1
OMK,1
OR01,1
OQW,1
OQN,1
OPN,1
OOK,1
ONX,1
ONO,1
ONA,1
OMH,1
OSC,1
OLV,1
OLS,1
OLG,1
OLF,1
OL99,1
OL70,1
OL68,1
OL60,1
OR29,1
OR42,1
OR44,1
OR48,1
OSA,1
ORT,1
ORS,1
ORI,1
ORB,1
OR92,1
OR85,1
OR84,1
OR82,1
OR81,1
OR80,1
OR79,1
OR77,1
OR76,1
OR67,1
OR62,1
OR60,1
PEZ,1
PFC,1
PGM,1
PUW,1
PYG,1
PXE,1
PWT,1
PWC,1
PVJ,1
PVF,1
PVE,1
PVC,1
PUJ,1
PHH,1
PTW,1
PTV,1
PTU,1
PTT,1
PTD,1
PTB,1
PSO,1
PSM,1
PYN,1
PYP,1
PZQ,1
R49,1
RFG,1
REI,1
REG,1
RED,1
RDV,1
RDR,1
RCR,1
RCP,1
RCK,1
RCA,1
RBY,1
RBO,1
RBM,1
RBL,1
RBG,1
RAW,1
RAS,1
PSK,1
PSG,1
PSE,1
PMZ,1
PMU,1
PMH,1
PMB,1
PLU,1
PLR,1
PLK,1
PLD,1
PKF,1
PKA,1
PJY,1
PJC,1
PIZ,1
PIM,1
PIB,1
PHT,1
PHO,1
PHK,1
PMV,1
PN39,1
PS42,1
PN43,1
PRS,1
PRN,1
PQL,1
PPQ,1
PPF,1
PPC,1
POV,1
POF,1
POE,1
POB,1
PNT,1
PNP,1
PNM,1
PNI,1
PNC,1
PNA,1
PN50,1
OL56,1
OL49,1
NE94,1
NYG,1
O19,1
O16,1
O15,1
O13,1
O09,1
O08,1
O05,1
O02,1
NY95,1
NV88,1
NY78,1
NY76,1
NY75,1
NY41,1
NY39,1
NY02,1
NY0,1
NXP,1
O20,1
O21,1
O24,1
O27,1
O57,1
O55,1
O54,1
O53,1
O52,1
O48,1
O47,1
O46,1
O45,1
O43,1
O41,1
O39,1
O37,1
O35,1
O33,1
O32,1
O28,1
NV99,1
NV68,1
OL48,1
NJM,1
NM20,1
NM2,1
NM18,1
NM16,1
NM12,1
NK35,1
NK05,1
NJW,1
NJ24,1
NV65,1
NHU,1
NH66,1
NGU,1
NGF,1
NFW,1
NFG,1
NFE,1
NE95,1
NM32,1
NM40,1
NOW,1
NQA,1
NV47,1
NV39,1
NV00,1
NUL,1
NUC,1
NSI,1
NRB,1
NR92,1
NR76,1
NR72,1
NR70,1
NR68,1
NR33,1
NR32,1
NR21,1
NR13,1
NR08,1
O59,1
O60,1
O61,1
OK48,1
OKH,1
OKB,1
OK94,1
OK93,1
OK90,1
OK69,1
OK66,1
OK56,1
OK36,1
O62,1
OK32,1
OK24,1
OK20,1
OK16,1
OK14,1
OJC,1
OJA,1
OIC,1
OKM,1
OKS,1
OKV,1
OKZ,1
OL47,1
OL46,1
OL44,1
OL42,1
OL41,1
OL40,1
OL39,1
OL38,1
OL36,1
OL32,1
OL31,1
OL30,1
OL29,1
OL28,1
OL24,1
OL23,1
OL18,1
OI98,1
OI85,1
OI41,1
OG39,1
OG13,1
OFP,1
OFF,1
OCH,1
OBU,1
OAR,1
OAJ,1
OA86,1
OA19,1
O89,1
O86,1
O85,1
O81,1
O79,1
O69,1
O68,1
O65,1
OG31,1
OGM,1
OI06,1
OGS,1
OH98,1
OH95,1
OH94,1
OH90,1
OH87,1
OH74,1
OH71,1
OH56,1
OH51,1
OH46,1
OH40,1
OH24,1
OH20,1
OH15,1
OH12,1
OH09,1
OH07,1
FYM,1
00AN,1
FYG,1
3MT9,1
3MO7,1
3MO5,1
3MN1,1
3M9,1
3M8,1
3M7,1
3M5,1
3M0,1
3LA8,1
3LA4,1
3L2,1
3KS9,1
3KS8,1
3KS7,1
3KS5,1
3KS4,1
3KS3,1
3KS2,1
3KS1,1
3MS1,1
3MY,1
5M5,1
3N6,1
3S4,1
3R9,1
3R8,1
3R4,1
3Q0,1
3PS2,1
3P3,1
3OK,1
3O8,1
3O5,1
3O4,1
3O3,1
3O1,1
3NY8,1
3NJ9,1
3NJ6,1
3NE5,1
3ND6,1
3NC6,1
3K8,1
3K7,1
3K4,1
3JA,1
3F6,1
3F4,1
3F3,1
3F2,1
3EX,1
3EV,1
3DW,1
3CK,1
3B4,1
3B3,1
3B0,1
3AU,1
3AK9,1
3A7,1
3A4,1
3A2,1
39Z,1
39TT,1
39SC,1
3F7,1
3FA7,1
3FK,1
3HO,1
3J6,1
3J1,1
3J0,1
3IS3,1
3ID3,1
3I7,1
3I4,1
3I2,1
3H5,1
3FL,1
3H4,1
3GA7,1
3G8,1
3G6,1
3G4,1
3G2,1
3FU,1
3FL0,1
3S5,1
3S6,1
3S7,1
40U,1
42ID,1
42IA,1
42I,1
42C,1
42B,1
42AK,1
42A,1
41XS,1
41U,1
41TT,1
41OH,1
41NR,1
41ND,1
41MS,1
41MO,1
41ME,1
41IL,1
41G,1
41F,1
42IN,1
42KS,1
42KY,1
43AS,1
43MN,1
43MI,1
43ID,1
43G,1
43FL,1
43D,1
43CN,1
43B,1
43A,1
42M,1
42XS,1
42XA,1
42VA,1
42U,1
42TT,1
42TS,1
42MT,1
42ME,1
40XS,1
40TN,1
3S8,1
40OR,1
3U5,1
3U4,1
3U2,1
3U1,1
3U0,1
3TN2,1
3TA1,1
3TA0,1
3T9,1
3T8,1
3T7,1
3T6,1
3T5,1
3T4,1
3T3,1
3T2,1
3T0,1
3SD4,1
3S9,1
3U7,1
3U9,1
3VA4,1
3X5,1
40MO,1
40G,1
40D,1
40AK,1
3Y7,1
3Y3,1
3Y2,1
3XA8,1
3WS9,1
3VG5,1
3WN1,1
3W9,1
3W7,1
3W6,1
3W5,1
3W3,1
3W2,1
3VS,1
39P,1
39N,1
39MT,1
2TA8,1
2WN2,1
2WI7,1
2WA2,1
2W6,1
2W5,1
2W3,1
2W2,1
2W1,1
2VA3,1
2V5,1
2UT9,1
2UT2,1
2UT,1
2U8,1
2U7,1
2U5,1
2U4,1
2U0,1
2TX,1
2WY8,1
2X2,1
2XS1,1
30K,1
31II,1
31ID,1
31GE,1
31E,1
30PA,1
30NE,1
30M,1
30KS,1
30IN,1
2XS2,1
30ID,1
30GE,1
30GA,1
30F,1
30AZ,1
2Z3,1
2Y4,1
2Y1,1
2TE2,1
2TA0,1
31VA,1
2T2,1
2PN1,1
2P7,1
2OK2,1
2OK1,1
2OH7,1
2OH6,1
2OA7,1
2O8,1
2O6,1
2O3,1
2O1,1
2NE8,1
2ND8,1
2N6,1
2N5,1
2N4,1
2MY5,1
2MU9,1
2MS3,1
2Q0,1
2R0,1
2R1,1
2S4,1
2T1,1
2SD8,1
2SD3,1
2S9,1
2S8,1
2S7,1
2S6,1
2S5,1
2S2,1
2R2,1
2S1,1
2S0,1
2RR,1
2R9,1
2R7,1
2R6,1
2R5,1
2R4,1
31MN,1
31WI,1
39MS,1
35TX,1
38AL,1
37V,1
37TX,1
37T,1
37S,1
37N,1
37MS,1
37MO,1
37IA,1
37F,1
37CN,1
37AA,1
36S,1
36NC,1
36MS,1
36ME,1
36KS,1
36IA,1
35XA,1
38C,1
38I,1
38IA,1
38XS,1
39MD,1
39KS,1
39K,1
39IL,1
39ID,1
39G,1
39B,1
39AR,1
38WT,1
38IL,1
38W,1
38VT,1
38U,1
38TT,1
38S,1
38N,1
38MS,1
38MN,1
35X,1
35TT,1
32A,1
35TE,1
33TE,1
33TA,1
33S,1
33R,1
33NR,1
33NC,1
33N,1
33M,1
33K,1
33J,1
33IA,1
33A,1
32WS,1
32WA,1
32S,1
32NR,1
32MT,1
32IN,1
32ID,1
33U,1
33VT,1
33WY,1
35AR,1
35S,1
35NR,1
35ME,1
35ID,1
35IA,1
35CN,1
35C,1
35AZ,1
35A,1
33XA,1
34XS,1
34SC,1
34R,1
34PA,1
34NY,1
34M,1
34IN,1
34G,1
43MO,1
43MT,1
43PA,1
53U,1
55MO,1
55M,1
55LA,1
55KS,1
55K,1
55I,1
55G,1
55D,1
54XS,1
54XA,1
54W,1
54TN,1
54NR,1
54MU,1
54MA,1
54LA,1
54KS,1
54CL,1
53XA,1
55MT,1
55NR,1
55S,1
56ME,1
57MI,1
57KS,1
57B,1
57AZ,1
56WV,1
56WA,1
56S,1
56MO,1
56I,1
55T,1
56FL,1
56F,1
56D,1
56C,1
56AL,1
55Y,1
55XS,1
55TT,1
53W,1
53T,1
57PN,1
53NR,1
51Z,1
51TT,1
51TE,1
51R,1
51MY,1
51MO,1
51ME,1
51M,1
51K,1
51J,1
51ID,1
51AS,1
51AL,1
51A,1
50S,1
50R,1
50PA,1
50OH,1
50KS,1
52A,1
52I,1
52J,1
52Y,1
53NH,1
53MT,1
53KT,1
53K,1
53IA,1
53GA,1
53AZ,1
53A,1
52U,1
52K,1
52TN,1
52S,1
52OA,1
52NR,1
52NK,1
52NH,1
52MO,1
52LA,1
57MO,1
57TS,1
50I,1
5D9,1
5J5,1
5IL7,1
5IL2,1
5IG9,1
5I6,1
5I2,1
5HO,1
5H4,1
5GA9,1
5GA3,1
5GA2,1
5G8,1
5G7,1
5G4,1
5G0,1
5FL9,1
5FL7,1
5F5,1
5F4,1
5J9,1
5K1,1
5K2,1
5KY5,1
5M1,1
5M0,1
5LS9,1
5LL7,1
5LA0,1
5L0,1
5KY7,1
5KY6,1
5KY4,1
5K4,1
5KY2,1
5KS9,1
5KS6,1
5KS4,1
5KS3,1
5KS2,1
5KS,1
5KE,1
5F0,1
5D6,1
57TX,1
5D2,1
59IA,1
59FD,1
59FA,1
59CO,1
59B,1
59AR,1
58XA,1
58TN,1
58TE,1
58T,1
58N,1
58MU,1
58MT,1
58MN,1
58M,1
58J,1
58C,1
58AR,1
57XA,1
59ID,1
59KS,1
59TN,1
5AZ9,1
5CL7,1
5C5,1
5C1,1
5B9,1
5B7,1
5B6,1
5B4,1
5B1,1
5AZ3,1
5A1,1
5AR8,1
5AR4,1
5AK,1
5A9,1
5A8,1
5A6,1
5A4,1
5A2,1
50K,1
50F,1
43SC,1
48P,1
4A5,1
4A4,1
4A3,1
4A2,1
49X,1
49OH,1
49NR,1
49MY,1
49MT,1
49MS,1
49ID,1
49G,1
49F,1
49C,1
49A,1
48X,1
48U,1
48TS,1
48TE,1
4A6,1
4A9,1
4AK,1
4B9,1
4F1,1
4E8,1
4D9,1
4D2,1
4C8,1
4C7,1
4C4,1
4C2,1
4B8,1
4AK1,1
4B6,1
4B1,1
4B0,1
4AR9,1
4AR4,1
4AR2,1
4AL8,1
4AK6,1
48PS,1
48NR,1
4F7,1
48MY,1
45K,1
45G,1
44XA,1
44U,1
44T,1
44SC,1
44NK,1
44N,1
44MO,1
44M,1
44G,1
44CL,1
44CA,1
44C,1
44B,1
44AR,1
44A,1
43XA,1
43TS,1
45KS,1
45MS,1
45R,1
47N,1
48LA,1
48K,1
48ID,1
48I,1
48G,1
48FA,1
47Y,1
47PA,1
47MT,1
46D,1
47KS,1
47K,1
47ID,1
47GE,1
47AR,1
46VA,1
46U,1
46KS,1
4F2,1
4F8,1
4Y9,1
4N1,1
4S1,1
4R9,1
4R7,1
4R6,1
4R5,1
4R3,1
4R1,1
4PS4,1
4PN8,1
4OR9,1
4OR1,1
4OK0,1
4O7,1
4O5,1
4O4,1
4O1,1
4NE0,1
4NC4,1
4N4,1
4S2,1
4S5,1
4S6,1
4V8,1
4Y8,1
4Y4,1
4XA2,1
4X4,1
4WN5,1
4W0,1
4VG3,1
4VA4,1
4V6,1
4S7,1
4V4,1
4V0,1
4U9,1
4U7,1
4TN9,1
4TA4,1
4T2,1
4S9,1
4N2,1
4MO5,1
4F9,1
4MO1,1
4KA,1
4K6,1
4K5,1
4K0,1
4J8,1
4J5,1
4J2,1
4J1,1
4I9,1
4I3,1
4GA5,1
4G8,1
4G7,1
4G5,1
4G4,1
4G3,1
4FA7,1
4FA2,1
4FA1,1
4KS,1
4KS4,1
4KS9,1
4MB,1
4MN7,1
4MN2,1
4MN1,1
4MI4,1
4ME7,1
4ME0,1
4MD8,1
4MD5,1
4M9,1
4KY8,1
4M8,1
4M7,1
4M5,1
4M4,1
4M2,1
4M1,1
4M0,1
4LL0,1
2MO8,1
2MO,1
2MN4,1
0R5,1
0U9,1
0U8,1
0U7,1
0U3,1
0U2,1
0U1,1
0U0,1
0TX7,1
0TX6,1
0TX1,1
0TE7,1
0TE5,1
0TE4,1
0S9,1
0S7,1
0S5,1
0S1,1
0S0,1
0R7,1
0V4,1
0V7,1
0VG,1
10KY,1
11A,1
10X,1
10WA,1
10U,1
10TT,1
10TS,1
10MO,1
10LS,1
10G,1
0W7,1
10D,1
10CO,1
0Y6,1
0XS2,1
0XA5,1
0XA4,1
0WI6,1
0WI5,1
0R6,1
0R4,1
11K,1
0R2,1
0M9,1
0M8,1
0M5,1
0M4,1
0M3,1
0M2,1
0M0,1
0L9,1
0L4,1
0KY1,1
0KS9,1
0KS8,1
0KS7,1
0KS6,1
0KS2,1
0K7,1
0J8,1
0J4,1
0J0,1
0MO3,1
0MO6,1
0MS0,1
0P2,1
0R1,1
0R0,1
0Q9,1
0Q5,1
0Q4,1
0PN0,1
0PA3,1
0P8,1
0P1,1
0MT6,1
0OH5,1
0O9,1
0O4,1
0O2,1
0NE9,1
0N4,1
0N0,1
0MU9,1
11IA,1
11KS,1
0IN,1
14Y,1
16TT,1
16TA,1
16S,1
16NY,1
16KS,1
16J,1
16G,1
16FL,1
16D,1
16A,1
15Z,1
15XA,1
15WA,1
15N,1
15MO,1
15M,1
15IA,1
15G,1
15F,1
16TX,1
16UT,1
16VG,1
17Q,1
18GE,1
18AZ,1
18AA,1
18A,1
17XS,1
17XA,1
17WT,1
17V,1
17N,1
16W,1
17M,1
17LS,1
17K,1
17J,1
17FD,1
17A,1
16X,1
16WI,1
15A,1
14TT,1
11N,1
14TS,1
13C,1
12Y,1
12V,1
12UT,1
12TT,1
12TA,1
12S,1
12ND,1
12N,1
12I,1
12D,1
12CN,1
12A,1
11V,1
11TT,1
11TN,1
11S,1
11R,1
11OR,1
13FD,1
13KS,1
13ME,1
14M,1
14TA,1
14S,1
14OI,1
14NC,1
14N,1
14MO,1
14ME,1
14MA,1
14LA,1
13PA,1
14J,1
14IA,1
14A,1
13Y,1
13W,1
13TT,1
13R,1
13Q,1
0IN4,1
0IL7,1
18IN,1
03TT,1
05MO,1
05MD,1
05LA,1
05KS,1
05K,1
05IA,1
05D,1
05C,1
05AN,1
05AK,1
04Y,1
04W,1
04V,1
04TT,1
04M,1
04G,1
04CA,1
04AZ,1
04AN,1
05N,1
05NR,1
05OH,1
06IA,1
06Y,1
06WN,1
06U,1
06TT,1
06TE,1
06N,1
06MO,1
06M,1
06FA,1
05S,1
06C,1
06AN,1
06AA,1
06A,1
05Y,1
05V,1
05UT,1
05U,1
04A,1
03S,1
07AN,1
03OL,1
01ID,1
01GE,1
01CN,1
01AN,1
01AA,1
01A,1
00XA,1
00W,1
00U,1
00TT,1
00TN,1
00S,1
00R,1
00N,1
00M,1
00IG,1
00F,1
00CL,1
00C,1
01J,1
01K,1
01M,1
02T,1
03MO,1
03M,1
03IA,1
03G,1
03FL,1
03D,1
03CO,1
03B,1
02NM,1
01TX,1
02MO,1
02MN,1
02MI,1
02G,1
02FD,1
02AN,1
02A,1
01WI,1
07A,1
07FA,1
0ID9,1
0A3,1
0CL3,1
0CD5,1
0CA0,1
0C8,1
0C5,1
0C4,1
0C2,1
0C1,1
0B7,1
0B5,1
0B2,1
0B1,1
0AZ9,1
0AZ1,1
0AR3,1
0AK,1
0AA4,1
0AA1,1
0A9,1
0CN1,1
0CN2,1
0D7,1
0GE3,1
0ID2,1
0ID,1
0IA4,1
0IA,1
0I8,1
0I2,1
0H7,1
0H1,1
0GA9,1
0E8,1
0G6,1
0G3,1
0FL0,1
0FD4,1
0F9,1
0F7,1
0F3,1
0F2,1
0A8,1
09Y,1
07GE,1
09UT,1
08KS,1
08IL,1
08C,1
08B,1
08AN,1
07Y,1
07V,1
07TS,1
07TE,1
07S,1
07R,1
07PA,1
07MT,1
07MD,1
07LA,1
07K,1
07ID,1
07IA,1
07I,1
08M,1
08N,1
08OL,1
09GE,1
09TS,1
09T,1
09S,1
09N,1
09MT,1
09M,1
09K,1
09J,1
09CL,1
08R,1
09AN,1
09A,1
08WI,1
08VA,1
08UT,1
08U,1
08TT,1
08TN,1
18I,1
18K,1
2MI4,1
24R,1
26U,1
26R,1
26NV,1
26N,1
26MO,1
26IN,1
26IL,1
26IA,1
26GA,1
26CL,1
26AS,1
26A,1
25U,1
25TX,1
25NV,1
25M,1
25J,1
25IA,1
25D,1
26W,1
27AK,1
27AZ,1
27PS,1
28ID,1
28DE,1
28AL,1
27XS,1
27WI,1
27W,1
27TX,1
27R,1
27P,1
27FA,1
27NR,1
27MS,1
27ME,1
27MD,1
27KS,1
27K,1
27ID,1
27IA,1
24S,1
24PS,1
28N,1
24NR,1
22XA,1
22S,1
22N,1
22MT,1
22MN,1
22M,1
22I,1
22F,1
22D,1
22C,1
21W,1
21UT,1
21TX,1
21TT,1
21ND,1
21N,1
21MU,1
21MS,1
21MO,1
23D,1
23JY,1
23M,1
24CO,1
24N,1
24MO,1
24LA,1
24K,1
24J,1
24ID,1
24IA,1
24F,1
24CL,1
23MI,1
24A,1
23W,1
23S,1
23R,1
23N,1
23MU,1
23MS,1
23MO,1
28M,1
28NE,1
21F,1
2G2,1
2J9,1
2J8,1
2J5,1
2J3,1
2J0,1
2IS,1
2IN,1
2IL9,1
2IL7,1
2IL3,1
2IA7,1
2I3,1
2I0,1
2H9,1
2H5,1
2H2,1
2GA7,1
2GA0,1
2G8,1
2K4,1
2K5,1
2K6,1
2L1,1
2MI3,1
2M7,1
2M4,1
2M2,1
2M1,1
2M0,1
2LL3,1
2LA7,1
2L0,1
2K8,1
2KY5,1
2KY0,1
2KS8,1
2KS6,1
2KS4,1
2KS3,1
2KL,1
2K9,1
2G4,1
2G1,1
28TT,1
2FD6,1
2AK,1
2A9,1
2A8,1
2A6,1
2A3,1
2A2,1
2A1,1
2A0,1
29OR,1
29N,1
29KY,1
29IL,1
29ID,1
29IA,1
29F,1
29D,1
29C,1
29AK,1
28U,1
2AK7,1
2AN,1
2AR4,1
2E6,1
2FA3,1
2FA1,1
2F7,1
2F4,1
2F1,1
2F0,1
2E8,1
2E7,1
2E2,1
2B2,1
2D7,1
2CL1,1
2C8,1
2C7,1
2C6,1
2C5,1
2B9,1
2B7,1
21M,1
20V,1
18KS,1
1E7,1
1I5,1
1I3,1
1H9,1
1H6,1
1H5,1
1H4,1
1H3,1
1H1,1
1GA0,1
1G8,1
1G4,1
1G3,1
1G1,1
1FL,1
1FD2,1
1F7,1
1F5,1
1F4,1
1F0,1
1I9,1
1IL4,1
1J9,1
1L3,1
1M4,1
1M2,1
1LA9,1
1LA5,1
1L9,1
1L8,1
1L7,1
1L4,1
1L1,1
1K1,1
1KS4,1
1KS,1
1K9,1
1K8,1
1K7,1
1K6,1
1K5,1
1K2,1
1E8,1
1E4,1
1M7,1
1E2,1
1A2,1
1A1,1
19TS,1
19NE,1
19N,1
19MT,1
19MO,1
19ME,1
19M,1
18Y,1
18TE,1
18T,1
18SC,1
18OI,1
18NV,1
18NR,1
18MS,1
18MO,1
18MI,1
1A3,1
1A4,1
1A5,1
1C1,1
1D8,1
1D4,1
1D3,1
1D2,1
1CO8,1
1C8,1
1C5,1
1C3,1
1BT,1
1A6,1
1B8,1
1B6,1
1B5,1
1B1,1
1AA0,1
1A9,1
1A8,1
1A7,1
1M5,1
1M8,1
20U,1
1TA7,1
1W1,1
1VA1,1
1V8,1
1V2,1
1V0,1
1UT9,1
1UT,1
1U9,1
1U6,1
1U4,1
1U3,1
1U2,1
1U1,1
1U0,1
1TX4,1
1TX0,1
1TN8,1
1TN0,1
1TE9,1
1W3,1
1WA,1
1WF,1
20A,1
20TT,1
20R,1
20OH,1
20N,1
20M,1
20KS,1
20K,1
20GA,1
1Z1,1
1X1,1
1Y9,1
1Y5,1
1Y3,1
1XS8,1
1XS4,1
1XS1,1
1XS0,1
1XA4,1
1TE4,1
1T8,1
1M9,1
1T7,1
1OR5,1
1OK,1
1OA2,1
1O8,1
1O6,1
1O5,1
1O4,1
1O2,1
1NY3,1
1NR7,1
1N7,1
1N3,1
1N1,1
1MU2,1
1MT4,1
1MS8,1
1MS2,1
1MO,1
1MN8,1
1P1,1
1PP,1
1Q1,1
1S2,1
1SC1,1
1S9,1
1S8,1
1S7,1
1S6,1
1S5,1
1S4,1
1S3,1
1S1,1
1Q2,1
1RL,1
1R8,1
1R7,1
1R4,1
1R1,1
1QK,1
1Q5,1
1Q4,1
5M4,1
5M6,1
FYE,1
BTP,1
BTF,1
BTA,1
BST,1
BSF,1
BRY,1
BRW,1
BQR,1
BQP,1
BQN,1
BQK,1
BPK,1
BOK,1
BNG,1
BMT,1
BMQ,1
BML,1
BMC,1
BLU,1
BLI,1
BTI,1
BTY,1
5M8,1
BUB,1
C05,1
C03,1
C02,1
C00,1
BYY,1
BYL,1
BYH,1
BYG,1
BXK,1
BXG,1
BXA,1
BWW,1
BWC,1
BVO,1
BVN,1
BVK,1
BVI,1
BUY,1
BUM,1
BLF,1
BKT,1
BKN,1
BKH,1
BCK,1
BCE,1
BCB,1
BBW,1
BBP,1
BBG,1
BBB,1
BAD,1
BAB,1
B21,1
B19,1
B10,1
B04,1
AZE,1
AZ90,1
AZ82,1
AZ72,1
AZ71,1
AZ66,1
BCR,1
BCT,1
BDG,1
BHC,1
BKF,1
BJJ,1
BIX,1
BIV,1
BIJ,1
BIF,1
BID,1
BHK,1
BGR,1
BDN,1
BGQ,1
BGF,1
BFZ,1
BFW,1
BFK,1
BFA,1
BEC,1
BDQ,1
C08,1
C09,1
C15,1
CEY,1
CKA,1
CJX,1
CJR,1
CJJ,1
CIK,1
CHU,1
CHQ,1
CHP,1
CHO,1
CHN,1
CGZ,1
CGS,1
CGF,1
CGE,1
CFT,1
CFK,1
CFJ,1
CFE,1
CFD,1
CKB,1
CKC,1
CKI,1
CLS,1
CN63,1
CN22,1
CN19,1
CN13,1
CMR,1
CMD,1
CMA,1
CLW,1
CLR,1
CKM,1
CLP,1
CL56,1
CL46,1
CL35,1
CL04,1
CKZ,1
CKX,1
CKU,1
CEZ,1
CEW,1
C17,1
CEU,1
C82,1
C74,1
C71,1
C66,1
C65,1
C63,1
C62,1
C61,1
C59,1
C57,1
C56,1
C55,1
C53,1
C44,1
C40,1
C28,1
C27,1
C25,1
C24,1
C88,1
C92,1
C97,1
CCB,1
CEM,1
CEA,1
CDK,1
CDI,1
CDH,1
CDA,1
CD05,1
CCO,1
CCA,1
C98,1
CBG,1
CAV,1
CAG,1
CA92,1
CA74,1
CA69,1
CA62,1
CA11,1
AZ57,1
AZ49,1
AZ19,1
A39,1
ADS,1
ADK,1
ADF,1
ACZ,1
ACQ,1
ACP,1
ACB,1
ABO,1
AAS,1
AAO,1
AA11,1
AA10,1
AA05,1
AA02,1
A79,1
A69,1
A68,1
A67,1
A63,1
ADU,1
AEJ,1
AFE,1
AJC,1
AK19,1
AK15,1
AK1,1
AK07,1
AK06,1
AK0,1
AJR,1
AJO,1
AIZ,1
AFJ,1
AIV,1
AHH,1
AHC,1
AGR,1
AGO,1
AFP,1
AFO,1
AFN,1
A61,1
A34,1
AK48,1
A32,1
9TX5,1
9TS3,1
9TE2,1
9S9,1
9S7,1
9S5,1
9S2,1
9S1,1
9OA5,1
9NR0,1
9NM9,1
9ND2,1
9N3,1
9MY7,1
9MT0,1
9MI4,1
9M8,1
9M6,1
9L2,1
9U1,1
9V5,1
9V6,1
9Y7,1
A30,1
A28,1
A26,1
A24,1
A20,1
A08,1
9Z9,1
9Z8,1
9Y2,1
9V7,1
9Y1,1
9Y0,1
9XS9,1
9XS4,1
9XA4,1
9WN6,1
9W9,1
9VA6,1
AK43,1
AK53,1
AYX,1
AOV,1
ASE,1
ASD,1
AS91,1
AS31,1
AS11,1
AS04,1
ARW,1
ARM,1
ARC,1
AQY,1
AQX,1
AQW,1
AQT,1
AQR,1
AQH,1
APY,1
APT,1
APS,1
APH,1
ASG,1
ASH,1
ASJ,1
AVX,1
AXX,1
AXV,1
AXS,1
AXQ,1
AXH,1
AWM,1
AWK,1
AWI,1
AVL,1
ASN,1
AVC,1
AUN,1
AUM,1
AUK,1
AUH,1
ATU,1
ATK,1
ASY,1
APG,1
AOH,1
AK59,1
AOC,1
AL09,1
AL05,1
AL04,1
AKW,1
AKR,1
AKQ,1
AKP,1
AKO,1
AKK,1
AKI,1
AKH,1
AKA,1
AK96,1
AK92,1
AK80,1
AK78,1
AK76,1
AK7,1
AK64,1
AL44,1
AL47,1
AL61,1
AN05,1
ANV,1
ANQ,1
ANP,1
ANK,1
ANJ,1
ANI,1
ANB,1
AN37,1
AN04,1
AL77,1
AN03,1
AN0,1
AMT,1
AMG,1
ALX,1
ALW,1
ALS,1
AL84,1
CN64,1
CN66,1
CN86,1
ELA,1
EUL,1
EUF,1
ETN,1
ETC,1
ESW,1
ERR,1
EQY,1
EPM,1
EOS,1
EOP,1
EOE,1
EOD,1
ENM,1
EMV,1
EMT,1
ELZ,1
ELO,1
ELN,1
ELK,1
EVU,1
EVW,1
EWU,1
F13,1
F34,1
F32,1
F31,1
F30,1
F23,1
F22,1
F17,1
F15,1
F10,1
EXX,1
F09,1
F08,1
F00,1
EZZ,1
EZM,1
EYW,1
EYF,1
EYE,1
ELI,1
EKY,1
F41,1
EKX,1
EBG,1
EBD,1
EBA,1
EAT,1
EAN,1
EAA,1
E98,1
E95,1
E94,1
E91,1
E89,1
E79,1
E78,1
E77,1
E75,1
E70,1
E68,1
E67,1
E63,1
ECP,1
ECS,1
ECU,1
EHM,1
EKS,1
EKQ,1
EKA,1
EIW,1
EIL,1
EIK,1
EHR,1
EHO,1
EGT,1
EDE,1
EGI,1
EGE,1
EET,1
EEO,1
EEK,1
EDU,1
EDN,1
EDJ,1
F35,1
F43,1
E58,1
FGU,1
FLV,1
FLT,1
FLP,1
FLG,1
FL83,1
FL75,1
FL74,1
FL59,1
FL37,1
FL36,1
FL34,1
FL03,1
FKS,1
FKN,1
FKA,1
FIT,1
FIG,1
FHR,1
FHAW,1
FME,1
FNB,1
FOM,1
FSO,1
FWQ,1
FWN,1
FWB,1
FVX,1
FVE,1
FUL,1
FSW,1
FSP,1
FSK,1
FOT,1
FSI,1
FSE,1
FRR,1
FRN,1
FRI,1
FRH,1
FQD,1
FOZ,1
FGX,1
FFT,1
F44,1
FFC,1
FA37,1
FA36,1
FA30,1
F99,1
F95,1
F92,1
F89,1
F88,1
F87,1
F86,1
F81,1
F70,1
F68,1
F62,1
F57,1
F53,1
F51,1
F47,1
F46,1
FA54,1
FA60,1
FA63,1
FCS,1
FFA,1
FET,1
FDW,1
FD90,1
FD38,1
FD23,1
FD03,1
FCY,1
FCI,1
FA86,1
FCH,1
FBK,1
FBG,1
FAQ,1
FAM,1
FAF,1
FA92,1
FA89,1
E60,1
E55,1
CN95,1
CSV,1
CXW,1
CXU,1
CXP,1
CXL,1
CXF,1
CXE,1
CXC,1
CWV,1
CWF,1
CWC,1
CVK,1
CVC,1
CVB,1
CUT,1
CUL,1
CUB,1
CTZ,1
CTJ,1
CT29,1
CYO,1
CYW,1
CZF,1
D11,1
D41,1
D32,1
D31,1
D29,1
D24,1
D20,1
D15,1
D14,1
D07,1
CZG,1
D06,1
D03,1
D02,1
D00,1
CZT,1
CZO,1
CZN,1
CZL,1
CT21,1
CSB,1
D47,1
CRZ,1
CO99,1
CO94,1
CO92,1
CO90,1
CO83,1
CO70,1
CO66,1
CO61,1
CO49,1
CO28,1
CO26,1
CO1,1
CO04,1
CNU,1
CNP,1
CNI,1
CNH,1
CNB,1
CN99,1
COD,1
COI,1
COM,1
CQM,1
CRX,1
CRW,1
CRT,1
CRQ,1
CRE,1
CRC,1
CQX,1
CQW,1
CQF,1
COT,1
CQB,1
CQA,1
CPX,1
CPU,1
CPT,1
CPP,1
CPK,1
CPC,1
D42,1
D49,1
E53,1
DQH,1
DXE,1
DWX,1
DWU,1
DWA,1
DVP,1
DVO,1
DUY,1
DUT,1
DUJ,1
DUH,1
DUC,1
DUB,1
DUA,1
DTS,1
DTG,1
DTA,1
DSV,1
DRT,1
DRP,1
DYA,1
DYB,1
DYL,1
E25,1
E52,1
E51,1
E45,1
E42,1
E36,1
E35,1
E34,1
E30,1
E24,1
DYT,1
E19,1
E16,1
E14,1
E13,1
E05,1
E02,1
DZJ,1
DZB,1
DRO,1
DPL,1
D51,1
DPG,1
DAA,1
D99,1
D98,1
D86,1
D83,1
D81,1
D76,1
D74,1
D73,1
D69,1
D66,1
D65,1
D64,1
D63,1
D58,1
D57,1
D56,1
D54,1
D52,1
DAF,1
DAW,1
DCK,1
DKR,1
DNA,1
DMW,1
DMA,1
DLZ,1
DLO,1
DLG,1
DLC,1
DKX,1
DIJ,1
DCU,1
DGL,1
DGG,1
DFI,1
DET,1
DEQ,1
DEH,1
DE25,1
DDH,1
9KY9,1
9KS9,1
9KS8,1
6S4,1
6Z6,1
6Y9,1
6Y5,1
6Y4,1
6Y3,1
6Y1,1
6XA4,1
6X0,1
6WI0,1
6WA3,1
6W4,1
6W0,1
6V5,1
6U7,1
6TX2,1
6TS8,1
6TE6,1
6SD4,1
6S9,1
70D,1
70J,1
70KS,1
72F,1
72XS,1
72XA,1
72TE,1
72S,1
72NH,1
72MI,1
72KS,1
72K,1
72AR,1
70MN,1
71TA,1
71LA,1
71K,1
71J,1
71AR,1
70TT,1
70S,1
70ND,1
6S6,1
6S3,1
73G,1
6S2,1
6K8,1
6K7,1
6K5,1
6K4,1
6K2,1
6J8,1
6J7,1
6J4,1
6J2,1
6J0,1
6IL7,1
6IL6,1
6ID1,1
6IA6,1
6I4,1
6I2,1
6H8,1
6H4,1
6G8,1
6KS,1
6KS4,1
6KS6,1
6MS6,1
6R7,1
6R3,1
6P7,1
6P3,1
6NE3,1
6NA6,1
6MU5,1
6MT4,1
6MN3,1
6KS7,1
6M8,1
6M7,1
6M6,1
6M2,1
6M0,1
6L5,1
6L4,1
6KS8,1
73C,1
73MD,1
6G4,1
77XS,1
7A3,1
7A0,1
79XA,1
79WS,1
79WA,1
79MU,1
79MT,1
79MI,1
79KY,1
79J,1
79D,1
79C,1
78Y,1
78XA,1
78VA,1
78R,1
78MO,1
78MI,1
78LA,1
7A5,1
7A6,1
7A8,1
7D9,1
7G2,1
7G0,1
7FL6,1
7FL3,1
7FD6,1
7FA3,1
7FA2,1
7F7,1
7D8,1
7AK,1
7B9,1
7B6,1
7B3,1
7B2,1
7AR9,1
7AR8,1
7AK6,1
7AK0,1
78AR,1
77XA,1
73MN,1
77TX,1
75G,1
75FL,1
75FA,1
75CO,1
75C,1
74V,1
74TN,1
74TE,1
74T,1
74S,1
74P,1
74ND,1
74N,1
74MU,1
74CA,1
74AR,1
73TS,1
73TA,1
73S,1
75J,1
75K,1
75KS,1
76N,1
77S,1
77NY,1
77MO,1
77MI,1
77KS,1
77F,1
76V,1
76T,1
76MY,1
75NC,1
76KS,1
76G,1
76F,1
76B,1
76A,1
75XS,1
75XA,1
75OH,1
6G5,1
6G1,1
7G9,1
5VA4,1
60KS,1
60J,1
60IA,1
60GE,1
60G,1
60F,1
60D,1
60C,1
60AS,1
60AR,1
5Z5,1
5Y7,1
5Y3,1
5Y2,1
5Y1,1
5WA5,1
5WA4,1
5WA2,1
5W5,1
60M,1
60MU,1
60OA,1
62K,1
63B,1
62XA,1
62TS,1
62S,1
62OH,1
62MA,1
62KT,1
62KS,1
61VA,1
60R,1
61S,1
61R,1
61KS,1
61K,1
61C,1
61A,1
60XA,1
60WN,1
5W4,1
5V5,1
63K,1
5V4,1
5OH6,1
5O1,1
5NN,1
5NK6,1
5NE3,1
5NC2,1
5NA0,1
5N8,1
5N4,1
5N2,1
5MU9,1
5MU5,1
5MS5,1
5MO7,1
5MO6,1
5MO,1
5MN1,1
5MI2,1
5MD4,1
5OI8,1
5P2,1
5P5,1
5SD3,1
5U8,1
5U6,1
5U1,1
5U0,1
5TN2,1
5TE0,1
5T9,1
5T4,1
5S6,1
5R1,1
5S4,1
5S2,1
5S1,1
5S0,1
5R8,1
5R7,1
5R4,1
5R2,1
63F,1
63KS,1
6FL8,1
67MI,1
69MU,1
69K,1
69G,1
68Y,1
68S,1
68R,1
68PN,1
68MN,1
68KS,1
68IS,1
68F,1
68D,1
67XA,1
67TT,1
67T,1
67SC,1
67S,1
67R,1
67MU,1
69N,1
69ND,1
69S,1
6B4,1
6F7,1
6F1,1
6E5,1
6D9,1
6D7,1
6D3,1
6D1,1
6CO5,1
6B0,1
6A2,1
6AR5,1
6AL5,1
6AL3,1
6AL0,1
6AK2,1
6A8,1
6A6,1
6A4,1
67MO,1
67MD,1
63M,1
67L,1
65J,1
65B,1
65AK,1
64TN,1
64S,1
64OG,1
64MS,1
64MN,1
64KS,1
64I,1
64GE,1
64G,1
64AR,1
63Y,1
63TN,1
63S,1
63NY,1
63MS,1
63ME,1
65MD,1
65MO,1
65MS,1
66R,1
67A,1
66Y,1
66XA,1
66V,1
66TN,1
66TE,1
66SC,1
66S,1
66OH,1
65S,1
66MO,1
66IL,1
66G,1
66AR,1
66AK,1
65XA,1
65WI,1
65TS,1
7G8,1
7GA2,1
9KS4,1
8U2,1
8Y5,1
8Y4,1
8XS9,1
8XA6,1
8WY6,1
8WV8,1
8WV3,1
8WI1,1
8WC,1
8W5,1
8W3,1
8W2,1
8VA9,1
8V7,1
8V6,1
8V3,1
8U8,1
8U6,1
8U4,1
8Y6,1
8Y8,1
90AR,1
91N,1
92KY,1
92IN,1
92G,1
92F,1
91XS,1
91WT,1
91TN,1
91NE,1
91ME,1
90F,1
91KS,1
91F,1
91C,1
91AS,1
90XA,1
90TT,1
90MN,1
90FD,1
8U3,1
8U1,1
93AK,1
8U0,1
8M9,1
8M7,1
8M2,1
8M1,1
8KS8,1
8KS5,1
8KS1,1
8KS0,1
8KS,1
8K0,1
8J7,1
8IL3,1
8IA8,1
8I3,1
8I1,1
8GK,1
8G8,1
8G6,1
8G3,1
8MI3,1
8MN4,1
8MO1,1
8S0,1
8TX0,1
8TE1,1
8TA9,1
8T8,1
8T6,1
8S4,1
8S2,1
8S1,1
8PA1,1
8MO4,1
8OH5,1
8OA7,1
8OA6,1
8NE6,1
8N8,1
8N4,1
8N2,1
8MY9,1
92MU,1
93B,1
8G1,1
97Y,1
9AK9,1
9AK7,1
9AK0,1
9A8,1
9A5,1
9A3,1
9A0,1
99V,1
99TT,1
99TA,1
99N,1
99MI,1
99KY,1
99CL,1
99A,1
98TS,1
98SC,1
98NH,1
98LS,1
9AL6,1
9AR1,1
9AR9,1
9G2,1
9KS3,1
9KS,1
9K2,1
9IS2,1
9I3,1
9I0,1
9G6,1
9G3,1
9G0,1
9B7,1
9F9,1
9F8,1
9D7,1
9D4,1
9D2,1
9D1,1
9D0,1
9C8,1
98KS,1
97MT,1
93F,1
97M,1
94Z,1
94TS,1
94TN,1
94R,1
94MI,1
94MD,1
94KS,1
94K,1
94IL,1
94FL,1
94FA,1
94C,1
93XA,1
93WA,1
93MS,1
93MO,1
93M,1
93KS,1
93FA,1
95D,1
95E,1
95F,1
96D,1
97KS,1
97FA,1
97CA,1
96Z,1
96TS,1
96NE,1
96KS,1
96G,1
96AL,1
95FA,1
96AK,1
95XS,1
95XA,1
95SC,1
95NE,1
95MO,1
95MI,1
95KS,1
8G2,1
8FL9,1
7GA7,1
7Q7,1
7W4,1
7W1,1
7VG4,1
7VA5,1
7V9,1
7V8,1
7V7,1
7V5,1
7V3,1
7V2,1
7U8,1
7TX0,1
7TA4,1
7T0,1
7S9,1
7S8,1
7S5,1
7S1,1
7S0,1
7W5,1
7W6,1
7XA2,1
80LA,1
81B,1
81AR,1
80XA,1
80T,1
80S,1
80OK,1
80OH,1
80MO,1
80KS,1
7XS0,1
80G,1
80F,1
80C,1
80AR,1
80AL,1
80A,1
7Y4,1
7Y3,1
7R9,1
7PN4,1
81KS,1
7PA4,1
7M3,1
7M2,1
7M1,1
7L8,1
7L2,1
7KY4,1
7KS9,1
7KS8,1
7KS6,1
7KS4,1
7KA,1
7K7,1
7K5,1
7J4,1
7IL3,1
7IA7,1
7I4,1
7I2,1
7GE7,1
7M4,1
7M5,1
7M6,1
7N4,1
7P5,1
7OR4,1
7OK7,1
7OH7,1
7NY8,1
7NE8,1
7NE5,1
7NC,1
7N1,1
7M7,1
7MY4,1
7MO8,1
7MO7,1
7MO1,1
7MN0,1
7MI,1
7ME,1
7M9,1
81FL,1
81ME,1
8FA9,1
87KS,1
88XS,1
88XA,1
88WS,1
88WA,1
88R,1
88PS,1
88PA,1
88OK,1
88MS,1
88MI,1
88MD,1
88M,1
88IN,1
88GA,1
88D,1
88C,1
87WS,1
87TN,1
87NR,1
89CA,1
89D,1
89MO,1
8C6,1
8FA1,1
8F6,1
8F5,1
8F3,1
8D8,1
8D4,1
8CD4,1
8CA8,1
8C4,1
89ND,1
8B2,1
8B1,1
8B0,1
8A7,1
8A3,1
89XA,1
89VA,1
89NR,1
87MS,1
87I,1
81OH,1
86OA,1
83KS,1
83K,1
83D,1
83B,1
82V,1
82TA,1
82NR,1
82NE,1
82ME,1
82J,1
82IS,1
82C,1
82AR,1
82A,1
81XA,1
81TS,1
81TN,1
81TE,1
81R,1
83MS,1
83MY,1
83NH,1
84TA,1
86NE,1
86ME,1
86F,1
86CL,1
86AK,1
85KS,1
85B,1
84TN,1
84R,1
83NR,1
84ME,1
84KY,1
84KS,1
84IL,1
84D,1
83XA,1
83TX,1
83Q,1
DNN,1

"""

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
from io import StringIO
import pandas as pd

def get_runways(flights_df):
    # Convert CSV string to Pandas DataFrame
    airport_runways = pd.read_csv(StringIO(csv_string))

    # Convert Pandas DF → Spark DF
    df_runways = spark.createDataFrame(airport_runways)

    return df_runways


com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

### Unweighted Betweenness Centrality

In [0]:
airport_info = {row['iata_code']: {'long': row['long'], 'lat': row['lat']} for row in df_airports.collect()}

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
import networkx as nx
import numpy as np
from pyspark.sql import Row

def get_unweighted_centralities(flight_df):
    # Collect edges into Python
    edges = flight_df.select("ORIGIN", "DEST").distinct().collect()

    G = nx.DiGraph()  # or nx.Graph() if your graph is undirected

    for row in edges:
        lat1, lon1 = airport_info[row["ORIGIN"]]["lat"], airport_info[row["ORIGIN"]]["long"]
        lat2, lon2 = airport_info[row["DEST"]]["lat"], airport_info[row["DEST"]]["long"]
        distance = haversine_np(lat1, lon1, lat2, lon2)
        G.add_edge(row["ORIGIN"], row["DEST"], weight=distance)

    bc = nx.betweenness_centrality(G, normalized=True, weight=None)
    cc = nx.closeness_centrality(G, distance='weight')

    bc_rows = [Row(ORIGIN=node, betweenness_unweighted=float(score)) for node, score in bc.items()]
    bc_df = spark.createDataFrame(bc_rows)

    cc_rows = [Row(ORIGIN=node, closeness=float(score)) for node, score in cc.items()]
    cc_df = spark.createDataFrame(cc_rows)


    return bc_df.join(cc_df, on='ORIGIN')


com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

### Weighted Betweenness




In [0]:
def get_weighted_centrality(flight_df):
    rows = flight_df.groupBy("ORIGIN", "DEST").agg(F.count("*").alias('num_flights')).collect()

    G = nx.DiGraph()

    for r in rows:
        # convert strength to cost
        cost = 1.0 / r["num_flights"]
        G.add_edge(r["ORIGIN"], r["DEST"], weight=cost)

    bc = nx.betweenness_centrality(G, weight="weight", normalized=True)



    bc_df = spark.createDataFrame([
        Row(ORIGIN=node, betweenness=float(score))
        for node, score in bc.items()
    ])

    return bc_df



com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
# Closeness centrality

# daily outgoing flights per runway (this is just degree/n_runways)
def average_delay_at_origin(flight_df):
    return flight_df.groupBy("ORIGIN").agg(F.avg("DEP_DELAY").alias('avg_origin_dep_delay'))

def average_delay_at_dest(flight_df):
    return flight_df.groupBy("DEST").agg(F.avg("ARR_DELAY").alias('avg_dest_arr_delay'))

# Other

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
# EDGE
# avg daily flights on route



def average_daily_flights_on_route(flight_df, DATASET_N_DAYS):
    return flight_df.groupBy("ORIGIN", "DEST").agg((F.count('*')/F.lit(DATASET_N_DAYS)).alias("avg_daily_route_flights"))
# average flight delay on route
def average_delay_on_route(flight_df):
    return flight_df.groupBy("ORIGIN", "DEST").agg(F.avg("DEP_DELAY").alias('avg_route_delay'))


# average flights at that current hour at that airport
def average_flights_at_hour(flight_df, DATASET_N_DAYS):
    return flight_df.groupBy(
        "ORIGIN",
        "DEST",
        "hour"
    ).agg(
        (F.count("*") / F.lit(DATASET_N_DAYS)).alias("avg_hourly_flights")
    )

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
vertex_functions_origin = [page_rank, out_degree, in_degree, weighted_in_out_degree, get_runways,
                           get_unweighted_centralities, get_weighted_centrality, average_delay_at_origin]
vertex_functions_dest = [average_delay_at_dest]


edge_functions = [average_daily_flights_on_route, average_delay_on_route]

edge_hourly_functions = [average_flights_at_hour]

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
def form_feature_df(functions, train_df, join_cols, DATASET_N_DAYS):
    assert len(functions) > 0
    try:
        df = functions[0](train_df, DATASET_N_DAYS)
    except:
        df = functions[0](train_df)
    for f in functions[1:]:
        try:
            df = df.join(f(train_df, DATASET_N_DAYS), on=join_cols)
        except:
            df = df.join(f(train_df), on=join_cols)

    return df

def join_features(input_df, origin_df, dest_df, edge_df, edge_hour_df):
    input_df = input_df.join(origin_df, on='ORIGIN')
    input_df = input_df.join(dest_df, on='DEST')
    input_df = input_df.join(edge_df, on=['ORIGIN', 'DEST'])
    input_df = input_df.join(edge_hour_df, on=['ORIGIN', 'DEST', 'hour'])
    return input_df

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
def add_features(dataset_dir):
    print(f"Starting run for {dataset_dir}")
    dataset_path = f"{checkpoint_path}/{dataset_dir}"
    graph_dir = 'graph_feature_splits'
    dbutils.fs.mkdirs(f"{dataset_path}/{graph_dir}")

    print("...Ingesting Data")
    train_df, validation_df, test_df = ingest_data(f"{dataset_path}/feature_eng_ph3/training_splits")    

    print("...Forming Features")
    DATASET_N_DAYS = get_dataset_days(train_df)

    origin_df = form_feature_df(vertex_functions_origin, train_df, ['ORIGIN'], DATASET_N_DAYS)

    dest_df = form_feature_df(vertex_functions_dest, train_df, ['DEST'], DATASET_N_DAYS)

    edge_df = form_feature_df(edge_functions, train_df, ['ORIGIN', 'DEST'], DATASET_N_DAYS)
        
    edge_hour_df = form_feature_df(edge_hourly_functions, train_df, ['ORIGIN', 'DEST', 'hour'], DATASET_N_DAYS)

    print("...Joining Features")
    train_df_joined = join_features(train_df, origin_df, dest_df, edge_df, edge_hour_df)
    validation_df_joined = join_features(validation_df, origin_df, dest_df, edge_df, edge_hour_df)
    test_df_joined = join_features(test_df, origin_df, dest_df, edge_df, edge_hour_df)

    print("...Writing features")
    
    train_df_joined.write.parquet(
        f"{dataset_path}/{graph_dir}/train",
        mode="overwrite"
    )
    validation_df_joined.write.parquet(
        f"{dataset_path}/{graph_dir}/validation",
        mode="overwrite"
    )
    test_df_joined.write.parquet(
        f"{dataset_path}/{graph_dir}/test",
        mode="overwrite"
    )

In [0]:
import time

checkpoint_path = "dbfs:/student-groups/Group_2_2"
dataset_dirs = ['3_month_custom_joined', '1_year_custom_joined', '5_year_custom_joined']

for dataset_dir in dataset_dirs:
    start_time = time.time()
    add_features(dataset_dir)
    elapsed = time.time() - start_time
    print(f"Completed {dataset_dir} in {elapsed/60.0:.2f} minutes")

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
checkpoint_path = "dbfs:/student-groups/Group_2_2"
display(dbutils.fs.ls(f"{checkpoint_path}"))

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
vertex_functions_origin = [page_rank, out_degree, in_degree, weighted_in_out_degree, get_runways,
                           get_unweighted_centralities, get_weighted_centrality, average_delay_at_origin]
vertex_functions_dest = [average_delay_at_dest]


edge_functions = [average_daily_flights_on_route, average_delay_on_route]

edge_hourly_functions = [average_flights_at_hour]


We used our training set to create 2 different categories of graph features: features of the origin airport, and features of the directed route from origin to destination. For origin features, we calculated weighted in and out degrees based on the number of ingoing and outgoing flights, page_rank, betweenness centrality, closeness centrality, and average delay at origin. For route features, we calculated the average daily route flights, average route delay, and average flights at that hour.



### Feature Descriptions

- Page Rank
- Weighted/Unweighted In/Out Degree

vertex_functions_origin = [page_rank, out_degree, in_degree, weighted_in_out_degree, get_runways,
                           get_unweighted_centralities, get_weighted_centrality, average_delay_at_origin]
vertex_functions_dest = [average_delay_at_dest]


edge_functions = [average_daily_flights_on_route, average_delay_on_route]

edge_hourly_functions = [average_flights_at_hour]